# Data Science 5 : Traitement d'Images avec Python et Filtres Morphologiques

Enseignant : Jean Delpech

Cours : Data Science

Classe : M1 Data/IA

Année scolaire : 2025/2026

Dernière mise à jour : janvier 2026

## Module Data Science M1 - Séances 11 & 12

**Objectifs du cours :**
- Rappels : Comprendre la représentation numérique des images (pixels, canaux, dimensions)
- Rappels : Maîtriser la manipulation d'images avec NumPy
- Rappels : Comprendre les espaces colorimétriques (RGB, niveaux de gris, HSV)
- Comprendre les fondements mathématiques de la morphologie mathématique
- Implémenter des filtres morphologiques "from scratch" avec NumPy
- Appliquer ces techniques à des cas d'usage réels

## Imports et configuration

In [ ]:
#!pip install jupyterlab-mathjax2=4.0.0 # installer s’il y a des pb dans l’affichage des formules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from PIL import Image

# Configuration
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.cmap'] = 'gray'
np.random.seed(42)

---

## 1. Représentation Numérique des Images

### 1.1 Qu'est-ce qu'une image numérique ?

Une **image numérique** est une matrice de valeurs numériques. Chaque élément est un **pixel**.

| Propriété | Description |
|-----------|-------------|
| **Hauteur (H)** | Nombre de lignes de pixels |
| **Largeur (W)** | Nombre de colonnes de pixels |
| **Canaux (C)** | 1 pour gris, 3 pour RGB |
| **Profondeur** | 8 bits → valeurs 0-255 |

Exemple de création d’une image en niveaux de gris (un seul canal) :

In [ ]:
# Image en niveaux de gris 5x5
img_gray_simple = np.array([
    [0,   0,   0,   0,   0],
    [0, 128, 128, 128,   0],
    [0, 128, 255, 128,   0],
    [0, 128, 128, 128,   0],
    [0,   0,   0,   0,   0]
], dtype=np.uint8)

print(f"Shape : {img_gray_simple.shape}")
print(f"Type : {img_gray_simple.dtype}")
print(f"Min/Max : {img_gray_simple.min()} / {img_gray_simple.max()}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(img_gray_simple, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Image')
axes[0].axis('off')

im = axes[1].imshow(img_gray_simple, cmap='gray', vmin=0, vmax=255)
for i in range(5):
    for j in range(5):
        color = 'white' if img_gray_simple[i, j] < 128 else 'black'
        axes[1].text(j, i, str(img_gray_simple[i, j]), ha='center', va='center', color=color)
axes[1].set_title('Valeurs des pixels')
axes[1].axis('off')
plt.tight_layout()
plt.show()

### 1.2 Images couleur RGB

Pour créer une image en couleur, selon la logique de la synthèse additive (cf. cours [Data-Science 01](./Data-Science-01-vecteurs-kmeans.ipynb), section illustration pratique des combinaisons linéaires), on va encoder l’image sur 3 canaux, c’est-à-dire 3 matrices de couleurs élémentaires, dont la combinaison va donner une couleur donnée à l’écran. Par exemple un pixel jaune sera un mélange de rouge et de vert :

In [ ]:
# Création d'une image RGB
img_rgb = np.zeros((100, 100, 3), dtype=np.uint8)
img_rgb[0:50, 0:50] = [255, 0, 0]      # Rouge
img_rgb[0:50, 50:100] = [0, 255, 0]    # Vert
img_rgb[50:100, 0:50] = [0, 0, 255]    # Bleu
img_rgb[50:100, 50:100] = [255, 255, 0] # Jaune

print(f"Shape RGB : {img_rgb.shape}")

fig, axes = plt.subplots(1, 4, figsize=(14, 3))
axes[0].imshow(img_rgb)
axes[0].set_title('Image RGB')
axes[0].axis('off')

for i, (titre, cmap) in enumerate([('Rouge', 'Reds'), ('Vert', 'Greens'), ('Bleu', 'Blues')]):
    axes[i+1].imshow(img_rgb[:, :, i], cmap=cmap, vmin=0, vmax=255)
    axes[i+1].set_title(f'Canal {titre}')
    axes[i+1].axis('off')
plt.tight_layout()
plt.show()

### 1.3 Image exemple pour le cours

Créons une image de manière procédurale, qui nous servira d’exemple pour le cours et nous permettra de bien voir les manipulation et transformations que nous allons implémenter :

In [ ]:
def creer_image_exemple():
    """Crée une image d'exemple."""
    h, w = 256, 256
    img = np.zeros((h, w, 3), dtype=np.uint8)
    
    # Dégradé
    for i in range(h):
        img[i, :, 0] = int(100 + 100 * np.sin(i * np.pi / h))
        img[i, :, 2] = int(100 + 100 * np.cos(i * np.pi / h))
    
    # Cercle
    y, x = np.ogrid[:h, :w]
    mask = (x - w//2)**2 + (y - h//2)**2 <= 60**2
    img[mask] = [255, 200, 100]
    
    # Rectangle
    img[30:80, 30:120] = [50, 180, 50]
    return img

img_exemple = creer_image_exemple()

print(f"Shape: {img_exemple.shape}, Type: {img_exemple.dtype}")
print(f"Pixels: {img_exemple.shape[0] * img_exemple.shape[1]:,}")

plt.figure(figsize=(6, 6))
plt.imshow(img_exemple)
plt.title('Image exemple')
plt.axis('off')
plt.show()

### Exercice 1 : Exploration d'une image

Un petit exercice simple pour vérfier que vous avez bien assimilé les concepts de base : indiquez les caractéristique de cette image.

In [ ]:
# EXERCICE - Votre code !

hauteur = #???  # Votre code
largeur = #???  
pixel_centre = #???  # np.array qui correspond au pixel au centre de l’image
moyenne_rouge = #???
moyenne_vert = #???
moyenne_bleu = #???

print(f"Dimensions : {hauteur} x {largeur}")
print(f"Pixel central : {pixel_centre}")
print(f"Moyennes : R={moyenne_rouge:.1f}, G={moyenne_vert:.1f}, B={moyenne_bleu:.1f}")

In [ ]:
# CORRIGÉ
hauteur = img_exemple.shape[0]
largeur = img_exemple.shape[1]
pixel_centre = img_exemple[hauteur // 2, largeur // 2]
moyenne_rouge = img_exemple[:, :, 0].mean()
moyenne_vert = img_exemple[:, :, 1].mean()
moyenne_bleu = img_exemple[:, :, 2].mean()

print(f"Dimensions : {hauteur} x {largeur}")
print(f"Pixel central : R={pixel_centre[0]}, G={pixel_centre[1]}, B={pixel_centre[2]}")
print(f"Moyennes : R={moyenne_rouge:.1f}, G={moyenne_vert:.1f}, B={moyenne_bleu:.1f}")

---

## 2. Conversion en Niveaux de Gris

### 2.1 Le retour de la combinaison linéaire

En data science, lorsque nous aurons à analyser ou traiter des images, la couleur n’est pas toujours une information pertinente. Cela dépend évidemment de la tâche précise à accomplir, mais par exemple pour de la reconnaissance de forme ou de la classification « en général », ainsi on arrive très bien à discriminer un chat d’un chien sur une image en noir et blanc.

Les images en noir et blanc (ou plus exactement « niveaux de gris ») étant codées sur un seul canal vs 3 canaux pour les images en couleur, il y a une différence significative en matière de ressources nécessaires au traitement d’une image en niveau de gris comparativement aux images en couleur (mémoire, temps de calcul, etc.)

Une transformation à laquelle on procédera souvent sera donc la conversion d’une image en couleur en niveaux de gris, passer de 3 canaux à un seul. Il est presque intuitif que pour parvenir à ce résultat en conservant une part de l’information, on peut simplement réaliser une combinaison linéaire des valeurs des 3 canaux :

$$
niveau_gris = \alpha · R + \beta · G + \gamma · B
$$

Mais quelle valeur choisir pour les coefficients $\alpha$, $\beta$ et $\gamma$ ?

Cela dépend de la « force » relative des canaux, de l’information que l’on veut conserver, et – encore une fois – de la tâche à accomplir.

Par exemple, pour un-e photographe, un-e graphiste, un-e monteur, un-e artiste numérique, etc., la dimension esthétique sera primordiale, et pour obtenir une image finale conforme à ses desseins, il mixera par exemple les canaux de manière à obtenir une image en niveau de gris bien contrastée, équilibrée, avec des noirs profonds et des blancs éclatants sans perdre d’information (préserver les détails dans les ombres et les « hautes lumières »).

Pour cela, il peut être intéressant de regarder comment son distribués les valeurs dans chaque canal, qu’il est courant de représenter sous forme d’histogramme (cf. des logiciels comme Photoshop ou Gimp) :

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].hist(img_exemple[:, :, 0].flatten(), bins=256, color='red')
axes[0].set_title('Histogramme canal rouge')

axes[1].hist(img_exemple[:, :, 1].flatten(), bins=256, color='green')
axes[1].set_title('Histogramme canal vert')

axes[2].hist(img_exemple[:, :, 2].flatten(), bins=256, color='blue')
axes[2].set_title('Histogramme canal bleu')

plt.show()


On constate que sur notre image :
* les pixels rouges :
    * ont en majorité des valeurs assez ressérées entre 100 et 200,
    * un grand nombre sont très rouges (avec une valeur précise au dessus de 250)
    * quelques uns sont faiblement rouge à 50
* les pixels verts :
    * la plupart ont une valeur de 0 (en réalité il n’y a que peu de pixels verts dans notre image)
    * deux valeurs minoritaire avec des valeurs précises en dessous de 200
* les pixels bleus :
    * ils sont ditribués de manière équilibrée entre 0 et 200
    * on distingue quelques pics (0, 50, 100 et 200)

Comment interpréter les caractéristiques de ces ditributions ?

* Pour le vert : on voit clairement que le vert ne figure pas parmi les dominantes de notre image. Il y a un rectangle uniformément vert, ce qui explique un des deux pics observés autour de 200, et un disque uniformément jaune (mélange de rouge et de vert), ce qui explique l’autre pic vert, ainsi que l’un des pics rouge (200 ou au dessus de 250). Pour le bleu, il est très présent dans le dégradé du fond, ce qui explique sa répartition équilibrée (avec la dominante rouge traduite par des valeurs plus ressérée), et notamment les couleurs violettes / pourpres sur le haut de l’image.

Une autre méthode consiste à suivre des **normes** ou **spécifications** : des espaces colorimétriques avec des caractéristiques définies précisément (souvent liées à une méthode d’affichage spécifique) peuvent définir des correspondances, par exemple pour la spécification Rec. 601, la pondération est la suivante :

$$
niveau_gris = 0.299 · R + 0.587 · G + 0.114 · B
$$

In [ ]:
def rgb_vers_gris(img_rgb):
    """Convertit RGB en niveaux de gris (Rec. 601)."""
    return (0.299 * img_rgb[:, :, 0] + 
            0.587 * img_rgb[:, :, 1] + 
            0.114 * img_rgb[:, :, 2]).astype(np.uint8)

img_gris = rgb_vers_gris(img_exemple)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img_exemple)
axes[0].set_title('RGB')
axes[0].axis('off')
axes[1].imshow(img_gris, cmap='gray')
axes[1].set_title('Niveaux de gris')
axes[1].axis('off')
plt.tight_layout()
plt.show()

Bien sûr nous avons détaillé là, pour votre culture générale de développeur, des considérations purement esthétiques (ou techniques, en matière d’étalonnage), ces considérations concernent assez peu le data scientist qui se contentera généralement d’un mixage uniforme des canaux, sauf besoins métiers ou situations particulières (vous saurez alors comment faire).

**Point important** : n’oubliez pas, lorsque vous calculez votre combinaison, de **normaliser** le résultat ! (par exemple, si vos pixels sont encodés en 8 bits, il faut notamment que la valeur maximum obtenue soit de 255).

### 2.2 Aller un peu plus loin avec les histogrammes

Commençons par créer une fonction qui permet d’afficher un histogramme d’une image en niveau de gris (ou d’un canal d’une image en couleur) avec quelques statistiques :

In [ ]:
def afficher_histogramme(image, titre="Histogramme", bins=256, afficher_stats=True, 
                         afficher_image=True, ax=None):
    """
    Affiche l'histogramme d'une image en niveaux de gris ou d'un canal couleur,
    avec optionnellement l'image elle-même à côté.
    
    Paramètres
    ----------
    image : ndarray
        Image 2D (niveaux de gris ou canal unique).
        Accepte uint8 (0-255) ou float (0-1).
    titre : str
        Titre du graphique.
    bins : int
        Nombre de bins pour l'histogramme (défaut: 256).
    afficher_stats : bool
        Si True, affiche moyenne, écart-type, min, max.
    afficher_image : bool
        Si True (défaut), affiche l'image à gauche de l'histogramme.
    ax : matplotlib.axes.Axes, optional
        Axes sur lesquels tracer l'histogramme. Si None, crée une nouvelle figure.
        Note: si afficher_image=True et ax=None, crée une figure avec 2 sous-graphiques.
              si afficher_image=True et ax fourni, l'image n'est pas affichée (ignoré).
    
    Retourne
    --------
    axes : matplotlib.axes.Axes ou tuple of Axes
        L'objet axes du graphique (ou tuple (ax_image, ax_hist) si afficher_image=True).
    
    Exemples
    --------
    >>> afficher_histogramme(image_gris)
    >>> afficher_histogramme(image_rgb[:, :, 0], titre="Canal Rouge")
    >>> afficher_histogramme(image_gris, afficher_image=False)  # histogramme seul
    """
    # Validation
    if image.ndim != 2:
        raise ValueError(f"L'image doit être 2D (reçu: {image.ndim}D). "
                        "Pour une image RGB, passez un canal: image[:, :, 0]")
    
    # Détection du type et normalisation
    if image.dtype == np.uint8:
        vmin, vmax = 0, 255
        xlabel = "Intensité (0-255)"
    elif np.issubdtype(image.dtype, np.floating):
        if image.max() <= 1.0:
            vmin, vmax = 0, 1
            xlabel = "Intensité (0-1)"
        else:
            vmin, vmax = 0, 255
            xlabel = "Intensité"
    else:
        vmin, vmax = image.min(), image.max()
        xlabel = "Intensité"
    
    # Création de la figure
    if ax is None:
        if afficher_image:
            fig, (ax_img, ax_hist) = plt.subplots(1, 2, figsize=(12, 4),
                                                   gridspec_kw={'width_ratios': [1, 1.3]})
        else:
            fig, ax_hist = plt.subplots(figsize=(10, 5))
            ax_img = None
    else:
        # Si ax est fourni, on l'utilise pour l'histogramme (pas d'image)
        ax_hist = ax
        ax_img = None
        if afficher_image:
            import warnings
            warnings.warn("afficher_image ignoré car ax est fourni. "
                         "Pour afficher l'image, ne passez pas ax.")
    
    # Affichage de l'image
    if ax_img is not None:
        ax_img.imshow(image, cmap='gray', vmin=vmin, vmax=vmax)
        ax_img.set_title("Image", fontsize=12, fontweight='bold')
        ax_img.axis('off')
    
    # Tracé de l'histogramme
    pixels = image.flatten()
    counts, bin_edges, patches = ax_hist.hist(pixels, bins=bins, range=(vmin, vmax), 
                                               color='steelblue', edgecolor='black', 
                                               alpha=0.7, linewidth=0.5)
    
    # Mise en forme
    ax_hist.set_xlabel(xlabel, fontsize=11)
    ax_hist.set_ylabel("Nombre de pixels", fontsize=11)
    ax_hist.set_title(titre, fontsize=12, fontweight='bold')
    ax_hist.set_xlim(vmin, vmax)
    ax_hist.grid(True, alpha=0.3, linestyle='--')
    
    # Statistiques
    if afficher_stats:
        moyenne = np.mean(pixels)
        ecart_type = np.std(pixels)
        val_min = np.min(pixels)
        val_max = np.max(pixels)
        
        # Ligne de moyenne
        ax_hist.axvline(x=moyenne, color='red', linestyle='-', linewidth=2, 
                        label=f'Moyenne: {moyenne:.1f}')
        
        # Zone ±1 écart-type
        ax_hist.axvspan(moyenne - ecart_type, moyenne + ecart_type, 
                        alpha=0.2, color='red', label=f'±σ: {ecart_type:.1f}')
        
        # Boîte de stats
        stats_text = (f"Moyenne: {moyenne:.1f}\n"
                     f"Écart-type: {ecart_type:.1f}\n"
                     f"Min: {val_min:.1f}\n"
                     f"Max: {val_max:.1f}")
        
        ax_hist.text(0.97, 0.95, stats_text, transform=ax_hist.transAxes, fontsize=10,
                     verticalalignment='top', horizontalalignment='right',
                     bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        ax_hist.legend(loc='upper left')
    
    plt.tight_layout()
    
    # Retour
    if ax_img is not None:
        return ax_img, ax_hist
    else:
        return ax_hist

In [ ]:
from skimage import data
img_chat = data.cat()
for i in range(3):
    print("="*10 + "\n")
    print(f"Canal {i+1}")
    afficher_histogramme(img_chat[:,:,i], titre="Image de chat")
    plt.show()

#### 2.2.1 Dynamique d’un histogramme

Les valeurs des pixels d’une image peuvent aller de 0 à 255 (en uint8). Pour différentes raisons, les images ne contiennent pas toujours des pixels qui contiennent toutes ces valeurs, pour des raisons esthétiques (faites une recherche high key/low key dans un moteur de recherches d’images) ou un problème d’exposition, de compression, etc. Une image qui contient un maximum d’information exploite la totalité de l’histogramme. Esthétiqueemnt, le fait qu’une photographie exploite toute la gamme de valeur est un critère notable (vous pouvez d’ailleurs voir que les images high/low key de qualité, si elles ont une dominante claire ou sombre, n’en contiennent pas moins des pixels avec toutes les valeurs, ce qui crée de beaux volumes par exemples). 

Nous pouvons explorer et modifier cette dynamique. Par exemples chargez l’image `images/histo-dyn-comp.jpg` : 

In [ ]:
from PIL import Image
import numpy as np

img = np.array(Image.open('images/histo-dyn-comp.jpg'))

Afficher les histogrammes :

In [ ]:
for i in range(3):
    afficher_histogramme(img[:,:,i], titre="Image dynamique histogramme")
    plt.show()

On constate que l’histogramme est complètement compressé sur la gauche (valeurs proches de zéro, de 0 à 8), et on n’affiche donc qu’une image noire.

Pour remédier à cela, il faut calculer une nouvelle image I’ avec des pixels dont les valeurs  s’étendent sur l’intervalle $[0, 255]$ en appliquant une transformation aux pixels de l’image I originale dont les valeurs appartiennent à $[0, 8]$.

On peut s’en sortir avec une simple transformation linéaire :

$$
I’(i, j) = \frac{I(i, j) - min_{i,j}I}{max_{i,j} I - min_{i,j} I} · (a - b) + a
$$

avec $a$ et $b$ l’intervalle voulu pour les valeurs des pixels de la nouvelle image.

Complétez la fonction suivante en implémentant la formule ci-dessus :

In [ ]:
def etirer_dynamique(image, a=0, b=255):
    """
    Étire la dynamique d'une image par transformation linéaire.
    
    Transforme les valeurs de pixels pour qu'elles occupent l'intervalle [a, b].
    
    Formule : I'(i,j) = (I(i,j) - min(I)) / (max(I) - min(I)) × (b - a) + a
    
    Paramètres
    ----------
    image : ndarray
        Image 2D en niveaux de gris.
    a : float
        Valeur minimale de l'intervalle de sortie (défaut: 0).
    b : float
        Valeur maximale de l'intervalle de sortie (défaut: 255).
    
    Retourne
    --------
    image_etiree : ndarray
        Image avec la dynamique étendue à [a, b].
        Même type que l'image d'entrée si uint8, sinon float.
    """
    img_float = image.astype(np.float64)
    
    val_min = img_float.min()
    val_max = img_float.max()
    
    if val_max == val_min:
        image_etiree = np.full_like(img_float, (a + b) / 2)
    else:
        image_etiree = (img_float - val_min) / (val_max - val_min) * (b - a) + a
    
    image_etiree = np.clip(image_etiree, a, b)
    
    if a == 0 and b == 255:
        return image_etiree.astype(np.uint8)
    else:
        return image_etiree

In [ ]:
# étirer la dynamique de l’image donnée en exemple
img_etiree = etirer_dynamique(img[:,:,0])
# afficher l’image et l’histogramme
afficher_histogramme(img_etiree, titre="Image - histogramme étiré");


On remarque néanmoins que cette méthode ne fait pas de miracle : on passe de 9 valeurs (de 0 à 8) à toujours 9 valeurs, mais étalées entre 0 et 255 (ce qui rend l’image lisible à un œil humain).

Dans l’exemple ci-dessus nous n’avons étiré l’histogramme d’un seul canal. Si vous voulez traiter une image en couleur, il faut bien sûr appliquer le traitement sur les 3 canaux (en prenant garde à la balance entre les couleurs). 

#### 2.2.3 Égalisation d’un histogramme

Nous pouvons généraliser la démarche vue à la section précédente, pour apporter toutes sortes de transformations à l’histogramme d’une image. 

Appliquer une transformation revient à créer une fonction qui met en correspondance une valeur donnée d’un ensemble de pixels de l’image d’origine avec une autre valeur pour les pixels correspondants de l’image transformée. Cette fonction de correspondance peut être représentée par ce que l’on va appeler une *lookup table* ou **LUT**. Une LUT peut représenter des fonctions linéaires ou non linéaires. Un bon exemple est l’outil `courbes` des logiciels de traitement d’image, comme *Photoshop* ou *Gimp* :

![L’outil courbes de Gimp](./images/LUT-histo.png)

Sur cette capture d’écran de l’outil `courbes` de Gimp, on distingue l’histogramme, en ordonnées les valeurs de sorties (image transformée) mises en correspondance avec les valeurs d’entrée en abscisse par la courbe que l’on peut modifier à la main (transforamtion non linéaire). On distingue également la droite $y = x$, qui est la transformation identité (pas de transformation), une transformation linéaire serait représentée par une droite de pente différente avec possiblement un décalage (offset). 

On peut distinguer des LUT avec un effet typique :

* $f(x) = 255 - x$ (inversion de la pente de la droite avec origine à 255) correspondra  à l’inversion (effet « négatif » ) des valeurs de l’image, c’est une transformation linéaire
* $f(x) = x + a$ avec $ a > 0$ correspondra à une augmentation uniforme (linéaire) de la luminosité (valeurs) de l’image (attention à écréter les valeurs à 255 ensuite). On peut bien sûr limiter cette augmentation de la luminosité à certains pixels (p. ex. les plus sombres) en posant des conditions d’application de la transformation.
* nous avons vu dans la section précédente l’étirement de la dynamique qui est une transformation linéaire également 
* l’égalisation d’histogramme qui est une transformation non-linéaire qui permet par exemple d’ajuster le contraste d’une image en répartissant plus uniformément les valeurs des pixels sur toute l’étendue des niveaux de gris (nous détaillons la formule ci-dessous)
* la correction gamma, qui est aussi une transformation linéaire à laquelle nous consacrons la section suivante

Pour revenir à l’égalisation, l'objectif de l'égalisation est de redistribuer les niveaux de gris pour que l'histogramme devienne approximativement uniforme, ce qui maximise le contraste de l'image (autant de valeurs sombres que de valeurs moyennes et de valeurs claires). Pour cela revenons aux concepts d’histogrammes normalisés et cumulés (revoyez votre cours sur les distributions) :

##### Histogramme normalisé $H(i)$

L'histogramme normalisé représente la probabilité qu'un pixel ait l'intensité $i$ :

$$
H(i) = \frac{\text{nombre de pixels d'intensité } i}{\text{nombre total de pixels}}
$$

Par exemple, si une image de 1000 pixels contient 50 pixels d'intensité 120, alors $H(120)=0.05$ (soit 5%).

##### Histogramme cumulé C(k)

$C(k)$ représente la probabilité qu'un pixel ait une intensité inférieure ou égale à $k$ :

$$
C(k) = \sum_{i=0}^{k} H(i)
$$

C'est la somme progressive des probabilités. Par construction :

* $C(0)=H(0)$ (probabilité d'avoir la valeur 0)
* $C(k)$ croît de façon monotone
* $C(255)=1$ (100% des pixels ont une valeur ≤ 255)

En statistiques, $C(k)$ correspond à la fonction de répartition (**CDF** ou *Cumulative Distribution Function*) de la distribution des intensités.

##### Fonction de transfert (LUT)

La LUT qui transforme chaque niveau de gris $i$ en sa nouvelle valeur $i′$ est simplement :

$$
f(i) = L \times C(i)
$$
où $L=255$ est le niveau de gris maximal.

La logique derrière cette formule est que la CDF $C(i)$ varie entre 0 et 1. 
En la multipliant par 255, on "étale" les valeurs sur toute la plage [0, 255]. Les niveaux de gris fréquents (où $C$ augmente rapidement) seront davantage espacés, tandis que les niveaux rares seront compressés. Le résultat est un histogramme approximativement uniforme.

In [ ]:
def egaliser_histogramme(image):
    """
    Égalise l'histogramme d'une image pour améliorer le contraste.
    
    L'égalisation redistribue les niveaux de gris pour que l'histogramme
    soit approximativement uniforme, maximisant ainsi le contraste.
    
    Principe :
    1. Calculer l'histogramme normalisé (probabilités)
    2. Calculer l'histogramme cumulé (CDF)
    3. Utiliser le CDF comme fonction de transformation
    
    Formule : I'(i,j) = (L-1) × CDF(I(i,j))
              où L = 256 (nombre de niveaux de gris)
    
    Paramètres
    ----------
    image : ndarray
        Image 2D en niveaux de gris (uint8).
    
    Retourne
    --------
    image_egalisee : ndarray (uint8)
        Image avec histogramme égalisé.
    cdf : ndarray
        Fonction de distribution cumulative (pour visualisation).
    """
    # Vérification
    if image.dtype != np.uint8:
        image = np.clip(image, 0, 255).astype(np.uint8)
    
    # 1. Histogramme (compte de pixels pour chaque niveau 0-255)
    hist, bins = np.histogram(image.flatten(), bins=256, range=(0, 256))
    
    # 2. Histogramme normalisé (probabilités)
    hist_norm = hist / hist.sum()
    
    # 3. CDF (Cumulative Distribution Function)
    cdf = np.cumsum(hist_norm)
    
    # 4. Transformation : utiliser le CDF comme LUT (Look-Up Table)
    # CDF va de 0 à 1, on le mappe vers [0, 255]
    lut = (cdf * 255).astype(np.uint8)
    
    # 5. Appliquer la transformation
    image_egalisee = lut[image]
    
    return image_egalisee, cdf

#### Exercice 2

Voici un exemple pas à pas d’égalisation d’histogramme sur une image simple :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def demo_egalisation_pas_a_pas():
    """
    Démonstration pas à pas de l'égalisation d'histogramme
    avec un exemple numérique simple.
    """
    
    # =========================================================================
    # EXEMPLE SIMPLIFIÉ : Image 4x4 avec seulement 8 niveaux de gris (0-7)
    # =========================================================================
    
    # Image exemple (4x4 = 16 pixels, niveaux 0-7)
    image_simple = np.array([
        [1, 2, 3, 2],
        [2, 3, 3, 4],
        [3, 4, 4, 5],
        [4, 5, 6, 6]
    ], dtype=np.uint8)
    
    L = 7  # Niveau max (on travaille sur 0-7 pour simplifier)
    n_pixels = image_simple.size  # 16 pixels
    
    print("=" * 70)
    print("ÉGALISATION D'HISTOGRAMME - EXEMPLE PAS À PAS")
    print("=" * 70)
    
    # -------------------------------------------------------------------------
    # ÉTAPE 1 : Compter les pixels (histogramme brut)
    # -------------------------------------------------------------------------
    print("\n" + "─" * 70)
    print("ÉTAPE 1 : Histogramme brut (comptage des pixels)")
    print("─" * 70)
    
    print("\nImage originale (4×4 pixels, niveaux 0-7) :")
    print(image_simple)
    
    # Compter manuellement
    hist_brut = np.zeros(L + 1, dtype=int)
    for i in range(L + 1):
        hist_brut[i] = np.sum(image_simple == i)
    
    print("\nComptage des pixels par niveau :")
    print("┌─────────┬" + "───────┬" * 7 + "───────┐")
    print("│ Niveau  │", end="")
    for i in range(L + 1):
        print(f"   {i}   │", end="")
    print()
    print("├─────────┼" + "───────┼" * 7 + "───────┤")
    print("│ Compte  │", end="")
    for i in range(L + 1):
        print(f"   {hist_brut[i]}   │", end="")
    print()
    print("└─────────┴" + "───────┴" * 7 + "───────┘")
    
    # -------------------------------------------------------------------------
    # ÉTAPE 2 : Histogramme normalisé H(i)
    # -------------------------------------------------------------------------
    print("\n" + "─" * 70)
    print("ÉTAPE 2 : Histogramme normalisé H(i) = compte(i) / total")
    print("─" * 70)
    
    H = hist_brut / n_pixels
    
    print(f"\nNombre total de pixels : {n_pixels}")
    print("\nCalcul de H(i) = compte(i) / 16 :")
    print()
    for i in range(L + 1):
        print(f"  H({i}) = {hist_brut[i]:2d} / 16 = {H[i]:.4f}")
    
    print("\n→ H(i) représente la PROBABILITÉ qu'un pixel ait l'intensité i")
    print(f"  Vérification : somme des H(i) = {H.sum():.4f} (doit être = 1)")
    
    # -------------------------------------------------------------------------
    # ÉTAPE 3 : Histogramme cumulé C(k)
    # -------------------------------------------------------------------------
    print("\n" + "─" * 70)
    print("ÉTAPE 3 : Histogramme cumulé C(k) = Σ H(i) pour i de 0 à k")
    print("─" * 70)
    
    C = np.cumsum(H)
    
    print("\nCalcul progressif de C(k) :")
    print()
    cumul = 0
    for k in range(L + 1):
        cumul += H[k]
        detail = " + ".join([f"H({i})" for i in range(k + 1)])
        print(f"  C({k}) = {detail}")
        print(f"       = {' + '.join([f'{H[i]:.3f}' for i in range(k+1)])}")
        print(f"       = {C[k]:.4f}")
        print()
    
    print("→ C(k) représente la probabilité qu'un pixel ait une intensité ≤ k")
    print("  C'est la FONCTION DE RÉPARTITION (CDF)")
    
    # -------------------------------------------------------------------------
    # ÉTAPE 4 : Fonction de transfert (LUT)
    # -------------------------------------------------------------------------
    print("\n" + "─" * 70)
    print("ÉTAPE 4 : LUT (Look-Up Table) : f(i) = L × C(i)")
    print("─" * 70)
    
    LUT = np.round(L * C).astype(np.uint8)
    
    print(f"\nAvec L = {L} (niveau max), on calcule f(i) = {L} × C(i) :")
    print()
    print("┌─────────┬──────────┬─────────────┬──────────────┬─────────────┐")
    print("│ Entrée  │   C(i)   │  L × C(i)   │   arrondi    │   Sortie    │")
    print("│    i    │          │             │              │    f(i)     │")
    print("├─────────┼──────────┼─────────────┼──────────────┼─────────────┤")
    for i in range(L + 1):
        produit = L * C[i]
        print(f"│    {i}    │  {C[i]:.4f}  │    {produit:.3f}    │      {LUT[i]}       │      {LUT[i]}       │")
    print("└─────────┴──────────┴─────────────┴──────────────┴─────────────┘")
    
    print("\n→ La LUT nous dit : 'le niveau i devient le niveau f(i)'")
    
    # -------------------------------------------------------------------------
    # ÉTAPE 5 : Application à l'image
    # -------------------------------------------------------------------------
    print("\n" + "─" * 70)
    print("ÉTAPE 5 : Application de la LUT à l'image")
    print("─" * 70)
    
    image_egalisee = LUT[image_simple]
    
    print("\nImage originale :          Image égalisée :")
    print()
    for row in range(4):
        orig_row = "  ".join([f"{v}" for v in image_simple[row]])
        egal_row = "  ".join([f"{v}" for v in image_egalisee[row]])
        print(f"    [{orig_row}]              [{egal_row}]")
    
    print("\nTransformations appliquées :")
    for i in range(L + 1):
        if hist_brut[i] > 0:
            print(f"  {i} → {LUT[i]}  ({hist_brut[i]} pixels)")
    
    # -------------------------------------------------------------------------
    # VISUALISATION
    # -------------------------------------------------------------------------
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # Ligne 1 : Avant
    axes[0, 0].imshow(image_simple, cmap='gray', vmin=0, vmax=L)
    axes[0, 0].set_title('Image originale', fontweight='bold')
    axes[0, 0].axis('off')
    for i in range(4):
        for j in range(4):
            axes[0, 0].text(j, i, str(image_simple[i, j]), ha='center', va='center',
                           color='red', fontsize=14, fontweight='bold')
    
    axes[0, 1].bar(range(L + 1), hist_brut, color='steelblue', edgecolor='black')
    axes[0, 1].set_title('Histogramme brut', fontweight='bold')
    axes[0, 1].set_xlabel('Niveau de gris')
    axes[0, 1].set_ylabel('Nombre de pixels')
    axes[0, 1].set_xticks(range(L + 1))
    
    axes[0, 2].bar(range(L + 1), H, color='teal', edgecolor='black')
    axes[0, 2].set_title('Histogramme normalisé H(i)', fontweight='bold')
    axes[0, 2].set_xlabel('Niveau de gris')
    axes[0, 2].set_ylabel('Probabilité')
    axes[0, 2].set_xticks(range(L + 1))
    
    axes[0, 3].bar(range(L + 1), C, color='darkorange', edgecolor='black')
    axes[0, 3].plot(range(L + 1), C, 'ro-', markersize=8)
    axes[0, 3].set_title('Histogramme cumulé C(k)\n(CDF)', fontweight='bold')
    axes[0, 3].set_xlabel('Niveau de gris')
    axes[0, 3].set_ylabel('Probabilité cumulée')
    axes[0, 3].set_xticks(range(L + 1))
    axes[0, 3].set_ylim(0, 1.1)
    
    # Ligne 2 : Après
    axes[1, 0].imshow(image_egalisee, cmap='gray', vmin=0, vmax=L)
    axes[1, 0].set_title('Image égalisée', fontweight='bold')
    axes[1, 0].axis('off')
    for i in range(4):
        for j in range(4):
            axes[1, 0].text(j, i, str(image_egalisee[i, j]), ha='center', va='center',
                           color='lime', fontsize=14, fontweight='bold')
    
    # LUT
    axes[1, 1].plot(range(L + 1), range(L + 1), 'k--', label='Identité', alpha=0.5)
    axes[1, 1].plot(range(L + 1), LUT, 'ro-', markersize=10, linewidth=2, label='LUT')
    axes[1, 1].set_title('LUT : f(i) = L × C(i)', fontweight='bold')
    axes[1, 1].set_xlabel('Entrée i')
    axes[1, 1].set_ylabel('Sortie f(i)')
    axes[1, 1].set_xticks(range(L + 1))
    axes[1, 1].set_yticks(range(L + 1))
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Histogramme après
    hist_egal, _ = np.histogram(image_egalisee, bins=L+1, range=(0, L+1))
    axes[1, 2].bar(range(L + 1), hist_egal, color='forestgreen', edgecolor='black')
    axes[1, 2].set_title('Histogramme après égalisation', fontweight='bold')
    axes[1, 2].set_xlabel('Niveau de gris')
    axes[1, 2].set_ylabel('Nombre de pixels')
    axes[1, 2].set_xticks(range(L + 1))
    axes[1, 2].axhline(y=n_pixels/(L+1), color='red', linestyle='--', 
                       label=f'Uniforme idéal ({n_pixels/(L+1):.1f})')
    axes[1, 2].legend()
    
    # Comparaison avant/après
    x = np.arange(L + 1)
    width = 0.35
    axes[1, 3].bar(x - width/2, hist_brut, width, label='Avant', color='steelblue')
    axes[1, 3].bar(x + width/2, hist_egal, width, label='Après', color='forestgreen')
    axes[1, 3].set_title('Comparaison avant/après', fontweight='bold')
    axes[1, 3].set_xlabel('Niveau de gris')
    axes[1, 3].set_ylabel('Nombre de pixels')
    axes[1, 3].set_xticks(range(L + 1))
    axes[1, 3].legend()
    
    plt.suptitle('Égalisation d\'histogramme - Exemple pas à pas', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # -------------------------------------------------------------------------
    # RÉSUMÉ FINAL
    # -------------------------------------------------------------------------
    print("\n" + "=" * 70)
    print("RÉSUMÉ")
    print("=" * 70)
    print("""
    AVANT : L'histogramme était concentré sur les valeurs 2-6
            → faible contraste, peu de niveaux utilisés
    
    APRÈS : L'histogramme est étalé sur 1-7
            → meilleur contraste, utilisation plus uniforme des niveaux
    
    La "magie" de l'égalisation :
    ─────────────────────────────
    • Les niveaux FRÉQUENTS (ex: 3, 4) → C(i) augmente vite → écartés
    • Les niveaux RARES (ex: 0, 1)     → C(i) augmente peu → compressés
    
    Résultat : les niveaux fréquents sont "étalés" pour occuper plus de place,
               ce qui augmente le contraste perçu.
    """)


if __name__ == "__main__":
    demo_egalisation_pas_a_pas()

Créons une image plus réaliste (plus de dégradés, formes…). Analysez l’histogramme de cette image puis égalisez le :

In [ ]:
# =========================================================================
# Création d'une image "sous-exposée" (faible contraste, sombre)
# =========================================================================
    
h, w = 256, 321  # 321 est divisible par 3
    
# Calcul explicite des bornes pour éviter les erreurs de broadcast
w1 = w // 3       # 107
w2 = 2 * w // 3   # 214
    
# Image avec plusieurs zones de textures
img = np.zeros((h, w), dtype=np.float64)
    
# Zone 1 : dégradé
for i in range(h):
    img[i, :w1] = 40 + 30 * (i / h)
    
# Zone 2 : texture bruit
zone2_width = w2 - w1
img[:, w1:w2] = np.random.normal(55, 15, (h, zone2_width))
    
# Zone 3 : formes géométriques
img[:, w2:] = 45
# Cercle clair
centre_x = w2 + (w - w2) // 2
for i in range(h):
    for j in range(w2, w):
        if (i - h//2)**2 + (j - centre_x)**2 < 50**2:
            img[i, j] = 80
# Rectangle sombre
img[h//4:h//4+60, w2+10:w2+70] = 30
    
# Limiter la dynamique à [25, 95] pour simuler une sous-exposition
img_complexe = np.clip(img, 25, 95).astype(np.uint8)

In [ ]:
afficher_histogramme(img_complexe, titre='Image complexe');

In [ ]:
def demo_egalisation_image_reelle(img):
    """
    Démonstration de l'égalisation d'histogramme sur une image réaliste.
    """
    
    # =========================================================================
    # Égalisation
    # =========================================================================
    
    L = 255
    
    # Histogramme et CDF
    hist, _ = np.histogram(img.flatten(), bins=256, range=(0, 256))
    hist_norm = hist / hist.sum()
    cdf = np.cumsum(hist_norm)
    
    # LUT
    lut = (L * cdf).astype(np.uint8)
    
    # Application
    img_egal = lut[img]
    
    # Histogramme après
    hist_egal, _ = np.histogram(img_egal.flatten(), bins=256, range=(0, 256))
    cdf_egal = np.cumsum(hist_egal / hist_egal.sum())
    
    # =========================================================================
    # Visualisation
    # =========================================================================
    
    fig = plt.figure(figsize=(16, 12))
    
    # Layout personnalisé
    gs = fig.add_gridspec(3, 4, height_ratios=[1, 1, 0.8], hspace=0.3, wspace=0.3)
    
    # --- Ligne 1 : Images ---
    ax_img1 = fig.add_subplot(gs[0, 0:2])
    ax_img1.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax_img1.set_title(f'Image originale\nDynamique : [{img.min()}, {img.max()}]', 
                      fontsize=12, fontweight='bold')
    ax_img1.axis('off')
    
    ax_img2 = fig.add_subplot(gs[0, 2:4])
    ax_img2.imshow(img_egal, cmap='gray', vmin=0, vmax=255)
    ax_img2.set_title(f'Image égalisée\nDynamique : [{img_egal.min()}, {img_egal.max()}]', 
                      fontsize=12, fontweight='bold')
    ax_img2.axis('off')
    
    # --- Ligne 2 : Histogrammes ---
    ax_hist1 = fig.add_subplot(gs[1, 0])
    ax_hist1.bar(range(256), hist, width=1, color='steelblue', edgecolor='none')
    ax_hist1.set_xlim(0, 255)
    ax_hist1.set_xlabel('Niveau de gris')
    ax_hist1.set_ylabel('Nombre de pixels')
    ax_hist1.set_title('Histogramme original', fontsize=11, fontweight='bold')
    ax_hist1.axvline(x=img.min(), color='red', linestyle='--', alpha=0.7)
    ax_hist1.axvline(x=img.max(), color='red', linestyle='--', alpha=0.7)
    
    ax_cdf1 = fig.add_subplot(gs[1, 1])
    ax_cdf1.plot(range(256), cdf, color='darkorange', linewidth=2)
    ax_cdf1.fill_between(range(256), cdf, alpha=0.3, color='darkorange')
    ax_cdf1.set_xlim(0, 255)
    ax_cdf1.set_ylim(0, 1)
    ax_cdf1.set_xlabel('Niveau de gris')
    ax_cdf1.set_ylabel('Probabilité cumulée')
    ax_cdf1.set_title('CDF original (= LUT normalisée)', fontsize=11, fontweight='bold')
    ax_cdf1.grid(True, alpha=0.3)
    # Montrer la zone "active"
    ax_cdf1.axvspan(img.min(), img.max(), alpha=0.2, color='red', label='Zone active')
    ax_cdf1.legend(fontsize=9)
    
    ax_hist2 = fig.add_subplot(gs[1, 2])
    ax_hist2.bar(range(256), hist_egal, width=1, color='forestgreen', edgecolor='none')
    ax_hist2.set_xlim(0, 255)
    ax_hist2.set_xlabel('Niveau de gris')
    ax_hist2.set_ylabel('Nombre de pixels')
    ax_hist2.set_title('Histogramme égalisé', fontsize=11, fontweight='bold')
    # Ligne de référence uniforme
    ax_hist2.axhline(y=img.size/256, color='red', linestyle='--', 
                     label='Uniforme idéal', alpha=0.7)
    ax_hist2.legend(fontsize=9)
    
    ax_cdf2 = fig.add_subplot(gs[1, 3])
    ax_cdf2.plot(range(256), cdf_egal, color='forestgreen', linewidth=2, label='CDF égalisé')
    ax_cdf2.plot(range(256), np.linspace(0, 1, 256), 'r--', linewidth=1, label='CDF uniforme idéal')
    ax_cdf2.set_xlim(0, 255)
    ax_cdf2.set_ylim(0, 1)
    ax_cdf2.set_xlabel('Niveau de gris')
    ax_cdf2.set_ylabel('Probabilité cumulée')
    ax_cdf2.set_title('CDF après égalisation', fontsize=11, fontweight='bold')
    ax_cdf2.legend(fontsize=9)
    ax_cdf2.grid(True, alpha=0.3)
    
    # --- Ligne 3 : LUT et explication ---
    ax_lut = fig.add_subplot(gs[2, 0:2])
    ax_lut.plot(range(256), range(256), 'k--', linewidth=1, alpha=0.5, label='Identité')
    ax_lut.plot(range(256), lut, color='purple', linewidth=2, label='LUT = 255 × CDF')
    ax_lut.set_xlim(0, 255)
    ax_lut.set_ylim(0, 255)
    ax_lut.set_xlabel('Niveau d\'entrée i', fontsize=11)
    ax_lut.set_ylabel('Niveau de sortie f(i)', fontsize=11)
    ax_lut.set_title('Look-Up Table (LUT)', fontsize=11, fontweight='bold')
    ax_lut.legend(fontsize=10)
    ax_lut.grid(True, alpha=0.3)
    ax_lut.set_aspect('equal')
    # Annotations
    ax_lut.annotate(f'min={img.min()} → {lut[img.min()]}', 
                    xy=(img.min(), lut[img.min()]), 
                    xytext=(img.min()+30, lut[img.min()]-40),
                    arrowprops=dict(arrowstyle='->', color='red'),
                    fontsize=10, color='red')
    ax_lut.annotate(f'max={img.max()} → {lut[img.max()]}', 
                    xy=(img.max(), lut[img.max()]), 
                    xytext=(img.max()+30, lut[img.max()]-40),
                    arrowprops=dict(arrowstyle='->', color='red'),
                    fontsize=10, color='red')
    
    # Explication textuelle
    ax_txt = fig.add_subplot(gs[2, 2:4])
    ax_txt.axis('off')
    
    plt.suptitle('Égalisation d\'histogramme - Image 256 niveaux', 
                 fontsize=14, fontweight='bold')
    plt.show()
    
    # =========================================================================
    # Stats
    # =========================================================================
    
    print("=" * 60)
    print("STATISTIQUES")
    print("=" * 60)
    print(f"""
    Image originale :
        Min / Max : {img.min()} / {img.max()}
        Moyenne   : {img.mean():.1f}
        Écart-type: {img.std():.1f}
    
    Image égalisée :
        Min / Max : {img_egal.min()} / {img_egal.max()}
        Moyenne   : {img_egal.mean():.1f}
        Écart-type: {img_egal.std():.1f}
    
    → L'écart-type a augmenté de {img.std():.1f} à {img_egal.std():.1f}
      ce qui traduit l'augmentation du contraste.
    """)
    
    conclusions = """
    INTERPRÉTATION DE LA LUT
    ━━━━━━━━━━━━━━━━━━━━━━━━
    
    • La LUT est plate pour les niveaux ABSENTS (0-24 et 96-255)
      → Ces niveaux ne sont pas utilisés dans l'image originale
    
    • La LUT monte RAPIDEMENT dans la zone [25-95]
      → Les niveaux présents sont "étalés" sur toute la plage [0-255]
    
    • Pente forte = niveaux fréquents → écartés (plus de contraste)
    • Pente faible = niveaux rares → compressés
    
    RÉSULTAT
    ━━━━━━━━
    • Avant : dynamique [25, 95] → 70 niveaux utilisés
    • Après : dynamique [0, 255] → 256 niveaux utilisés
    • Le contraste est maximisé !
    """

    print(conclusions)

demo_egalisation_image_reelle(img_complexe)

#### 2.2.3 Correction Gamma

La correction gamma est une transformation non-linéaire qui permet d'ajuster la luminosité d'une image de manière plus naturelle qu'une simple multiplication. Elle est définie par :
$$
I'(i,j) = 255 \times \left(\frac{I(i,j)}{255}\right)^\gamma
$$
Le paramètre γ contrôle la forme de la courbe de transformation :

γ = 1 : fonction identité, l'image reste inchangée
γ < 1 : éclaircit l'image, en relevant particulièrement les zones sombres
γ > 1 : assombrit l'image, en compressant les valeurs claires

L'intérêt de cette transformation réside dans son comportement asymétrique : elle affecte davantage certaines plages de valeurs que d'autres. Avec γ < 1, les pixels sombres (proches de 0) sont fortement relevés tandis que les pixels clairs (proches de 255) changent peu. C'est l'inverse avec γ > 1.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

  
x = np.linspace(0, 255, 256)
x_norm = x / 255  # Normalisation [0, 1]
    
gammas = [0.3, 0.5, 0.7, 1.0, 1.5, 2.2, 3.0]
couleurs = ['#e41a1c', '#ff7f00', '#ffd700', '#000000', '#4daf4a', '#377eb8', '#984ea3']
    
fig, ax = plt.subplots(figsize=(16, 5))
    
# --- Courbes de transformation ---
for gamma, couleur in zip(gammas, couleurs):
    y = 255 * (x_norm ** gamma)
    style = '-' if gamma != 1.0 else '--'
    lw = 2 if gamma != 1.0 else 3
    ax.plot(x, y, style, color=couleur, linewidth=lw, label=f'γ = {gamma}')
    
ax.set_xlim(0, 255)
ax.set_ylim(0, 255)
ax.set_xlabel('Valeur d\'entrée', fontsize=11)
ax.set_ylabel('Valeur de sortie', fontsize=11)
ax.set_title('Courbes de correction gamma', fontsize=12, fontweight='bold')
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
    
# Annotations zones
ax.fill_between([0, 80], [0, 0], [255, 255], alpha=0.1, color='blue')
ax.fill_between([180, 255], [0, 0], [255, 255], alpha=0.1, color='red')
ax.text(40, 240, 'Sombres', ha='center', fontsize=10, color='blue')
ax.text(217, 240, 'Clairs', ha='center', fontsize=10, color='red')
    
# =========================================================================
# Application sur un dégradé
# =========================================================================
    
np.random.seed(42)
    
# Image test avec gradient et formes
h, w = 150, 400
img = np.zeros((h, w), dtype=np.uint8)
    
# Gradient horizontal
for j in range(w):
    img[:, j] = int(255 * j / w)
    
# Appliquer différents gamma
gammas_demo = [0.4, 0.7, 1.0, 1.5, 2.5]
    
fig, axes = plt.subplots(len(gammas_demo), 2, figsize=(14, 10),
                              gridspec_kw={'width_ratios': [2, 1]})
    
for idx, gamma in enumerate(gammas_demo):
    # Correction gamma
    img_gamma = (255 * ((img / 255) ** gamma)).astype(np.uint8)
        
    # Image
    axes[idx, 0].imshow(img_gamma, cmap='gray', vmin=0, vmax=255)
    if gamma < 1:
        effet = "éclaircit"
        couleur = 'darkorange'
    elif gamma > 1:
        effet = "assombrit"
        couleur = 'steelblue'
    else:
        effet = "identité"
        couleur = 'black'
    axes[idx, 0].set_title(f'γ = {gamma} ({effet})', fontsize=11, 
                                fontweight='bold', color=couleur)
    axes[idx, 0].axis('off')
        
    # Histogramme
    axes[idx, 1].hist(img_gamma.flatten(), bins=256, range=(0, 256),
                          color=couleur, alpha=0.7, edgecolor='none')
    axes[idx, 1].set_xlim(0, 255)
    axes[idx, 1].set_ylabel('Pixels')
    if idx == len(gammas_demo) - 1:
            axes[idx, 1].set_xlabel('Intensité')
    axes[idx, 1].grid(True, alpha=0.3)
    
plt.suptitle('Effet de la correction gamma sur un dégradé', 
                 fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
def correction_gamma(image, gamma):
    """
    Applique une correction gamma à une image.
    
    La correction gamma est une transformation non-linéaire qui permet
    d'ajuster la luminosité de manière perceptuellement uniforme.
    
    Formule : I'(i,j) = 255 × (I(i,j) / 255)^gamma
    
    Paramètres
    ----------
    image : ndarray
        Image 2D en niveaux de gris.
    gamma : float
        Exposant gamma.
        - gamma < 1 : éclaircit l'image (relève les ombres)
        - gamma = 1 : image inchangée
        - gamma > 1 : assombrit l'image (accentue les contrastes sombres)
    
    Retourne
    --------
    image_corrigee : ndarray (uint8)
        Image avec correction gamma appliquée.
    """
    # Normaliser vers [0, 1]
    img_norm = image.astype(np.float64) / 255.0
    
    # Appliquer gamma
    img_gamma = np.power(img_norm, gamma)
    
    # Revenir vers [0, 255]
    image_corrigee = (img_gamma * 255).astype(np.uint8)
    
    return image_corrigee

In [ ]:
img_foncee = img_chat[:,:,2]
# correction linéaire
a = 50
img_lin = img_chat[:,:,2] + a

img_gamma = correction_gamma(img_foncee, 0.65)
fig, axes = plt.subplots(1, 3, figsize=(12, 6))

axes[0].imshow(img_foncee)
axes[0].set_title('Image d’origine (foncée)', fontsize=11, fontweight='bold')

axes[1].imshow(img_gamma)
axes[1].set_title('Image corrigée (éclaircie avec γ = 0.65)', fontsize=11, fontweight='bold')

axes[2].imshow(img_lin)
axes[2].set_title('Image corrigée (éclaircie linéairement a = 50)', fontsize=11, fontweight='bold')

fig.suptitle('Correction gamma sur une image « réelle »', 
                 fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

On constate que l’éclaircicement obtenu avec la correction gamma préserve mieux les contrastes (image plus équilibrée). Testez et comparez différentes valeurs de γ et de $a$.


## 3. Seuillage

### 3.1 Seuillage simple

Une fois notre image convertie en niveau de gris, nous pouvons procéder à d’autres traitements, afin d’isoler, ou faciliter l’isolement de certains éléments (p. ex. ce que l’on appelle la segmentation) ou au contraire de les faire disparaître (en général pour nettoyer l’image : supprimer le bruit ou des artefact, etc.).

Un exemple classique : le traitement des empreintes digitales. L’information utile dans une emprunte sont les motifs dessinés par les sillons, il n‘y a aucun intérêt à analyser des nuances de gris etc. Le seuillage permet d’extraire ces motifs :

![photo empreinte digitale Wikimedia](./images/Fingerprintonpaper.jpg)
CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=618743

L’approche la plus simple (ou plutôt simpliste) consiste à effectuer un seuillage : on partitionne notre espace de valeurs en deux en fixant un seuil (*threshold*) $T$, c’est à dire la valeur limite à partir de laquelle le « gris » sera considéré comme du blanc, et en dessous de laquelle il sera considéré comme du noir. 

$$
I'(x,y) = \begin{cases} 255 & \text{si } I(x,y) > T \\ 0 & \text{sinon} \end{cases}
$$

On transforme donc une image en niveau de gris en une image binaire, où une valeur de pixel (p. ex. le noir) pourra par exemple représenter le fond, et l’autre valeur, le premier plan. En fixant adroitement notre seuil, on peut isoler certains éléments de notre image :

In [ ]:
def seuillage_simple(img_gris, seuil):
    """Seuillage binaire."""
    return np.where(img_gris > seuil, 255, 0).astype(np.uint8)

seuils = [50, 100, 150, 200]
fig, axes = plt.subplots(1, 5, figsize=(16, 3))
axes[0].imshow(img_gris, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

for i, s in enumerate(seuils):
    axes[i+1].imshow(seuillage_simple(img_gris, s), cmap='gray')
    axes[i+1].set_title(f'Seuil = {s}')
    axes[i+1].axis('off')
plt.tight_layout()
plt.show()

#### Exercice 3

Appliquez cette fonction sur l’image d’emprunte digitale que vous aurez chargée (chemin : `./images/Fingerprintonpaper.jpg`. Comparez plusieurs valeurs de seuil.

In [ ]:
# EXERCICE 3 : VOTRE CODE ! 
fingerprint = np.array(Image.open('images/Fingerprintonpaper.jpg'))
fingerprint_treated = seuillage_simple(fingerprint, 140)
plt.imshow(fingerprint_treated)
plt.show()

Les limites de cette approche sont évidentes : il nous faut une méthode pour déterminer la valeur seuil, des éléments différents peuvent avoir des niveaux identiques et donc seront traité comme un tout, etc. On peut bien sûr imaginer plusieurs seuils distincts pour segmenter notre image (par exemple ici avec deux niveaux on pourrait séparer le fond, le rectange et le disque), mais cela ne règle le problème qu’en partie et surtout ne résout pas la question : comment fixer ces seuils autrement que par essai/erreur.

Comme nous sommes des data scientists nous allons d’abord chercher un moyen d’automatiser la procédure à partir des informations contenues dans l’image.

Note : on applique généralement les seuils aux niveaux de gris (un seuil = un canal), mais on peut traiter un canal de couleur de la même manière, même si en pratique l’intérêt est souvent nul.

### 3.2 Seuillage d'Ōtsu (automatique)

Cette méthode, comme son nom l’indique, a été inventée par *Nobuyuki Ōtsu* en 1979. Elle va chercher à fixer le seuil à partir de la distribution des valeurs des pixels (donc de l’histogramme), donc à partir des valeurs rencontrées dans l’image elle-même (et pas arbitrairement), ce qui permet d’adapter automatiquement le seuil à une image donnée. 

Le principe est somme toute assez simple : on va chercher une valeur limite (seuil) qui va minimiser la variance intra-classe des deux classes de valeurs des pixels qui constituent l’image finale.

Plus précisément :

* chaque pixel de l’image finale binaire a une valeur parmi deux possibles (0 ou 255 par exemple pour noir / blanc), ce qui définit deux classes
* pour affecter les pixels de l’image d’origine à l’une ou l’autre classe, on va regarder s’ils sont supérieurs ou inférieurs à une valeur $S$ (seuil)
* par exemple si on fixe le seuil à 124, la classe 0 va contenir l’ensemble des pixels dont les valeurs sont comprises entre 0 et 124, et la classe 255 tous les pixels de valeurs supérieures à 124
* dans chacunes de ces classes, en fonction de l’image d’origine, on va retrouver des distributions différentes de valeurs
* selon Ōtsu, le meilleur seuil est celui qui crée deux groupes les plus homogènes possibles, car une image qui peut être seuillée « efficacement » est une image dont l’histogramme contient deux distributions homogènes (p. ex. un histogramme qui montre une distribution bimodale. Si on « coupe » au bon endroit on devrait obtenir des classes qui ne contiennent que des pixels aux valeurs « similaires », avec une faible dispersion

Voyons cela avec un histogramme créé pour l’exemple :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# =============================================================================
# CRÉATION DES DONNÉES (même distribution bimodale)
# =============================================================================

np.random.seed(42)

# Groupe 1 : pixels sombres (fond), centrés autour de 60
pixels_fond = np.random.normal(60, 20, 3000).astype(int)
pixels_fond = np.clip(pixels_fond, 0, 255)

# Groupe 2 : pixels clairs (objets), centrés autour de 180
pixels_objets = np.random.normal(180, 25, 2000).astype(int)
pixels_objets = np.clip(pixels_objets, 0, 255)

# Tous les pixels
tous_pixels = np.concatenate([pixels_fond, pixels_objets])

# =============================================================================
# CALCUL DE LA VARIANCE INTRA-CLASSE POUR CHAQUE SEUIL
# =============================================================================

def calculer_variance_intra_classe(pixels, seuil):
    """
    Calcule la variance intra-classe pondérée pour un seuil donné.
    
    Formule : σ²_intra = w0 × σ²_0 + w1 × σ²_1
    
    où :
    - w0, w1 = proportions de pixels dans chaque classe
    - σ²_0, σ²_1 = variances de chaque classe
    """
    classe0 = pixels[pixels <= seuil]
    classe1 = pixels[pixels > seuil]
    
    # Éviter les classes vides
    if len(classe0) == 0 or len(classe1) == 0:
        return np.inf
    
    # Proportions (poids)
    w0 = len(classe0) / len(pixels)
    w1 = len(classe1) / len(pixels)
    
    # Variances de chaque classe
    var0 = np.var(classe0)
    var1 = np.var(classe1)
    
    # Variance intra-classe pondérée
    variance_intra = w0 * var0 + w1 * var1
    
    return variance_intra

def calculer_variance_inter_classe(pixels, seuil):
    """
    Calcule la variance inter-classe pour un seuil donné.
    
    Formule : σ²_inter = w0 × w1 × (μ0 - μ1)²
    
    C'est cette quantité qu'Otsu MAXIMISE (équivalent à minimiser σ²_intra)
    """
    classe0 = pixels[pixels <= seuil]
    classe1 = pixels[pixels > seuil]
    
    if len(classe0) == 0 or len(classe1) == 0:
        return 0
    
    w0 = len(classe0) / len(pixels)
    w1 = len(classe1) / len(pixels)
    
    mu0 = np.mean(classe0)
    mu1 = np.mean(classe1)
    
    variance_inter = w0 * w1 * (mu0 - mu1) ** 2
    
    return variance_inter

# Calculer pour tous les seuils possibles
seuils = np.arange(1, 255)
variances_intra = [calculer_variance_intra_classe(tous_pixels, t) for t in seuils]
variances_inter = [calculer_variance_inter_classe(tous_pixels, t) for t in seuils]

# Trouver le seuil optimal
seuil_otsu = seuils[np.argmin(variances_intra)]
variance_min = min(variances_intra)

print(f"Seuil optimal (Otsu) : {seuil_otsu}")
print(f"Variance intra-classe minimale : {variance_min:.1f}")

# =============================================================================
# VISUALISATION COMPLÈTE
# =============================================================================

fig = plt.figure(figsize=(15, 10))

# Layout : 2 lignes, avec la première divisée en 2
ax1 = fig.add_subplot(2, 2, 1)  # Histogramme
ax2 = fig.add_subplot(2, 2, 2)  # Variance intra-classe
ax3 = fig.add_subplot(2, 1, 2)  # Explication visuelle avec 3 seuils

# --- 1. Histogramme avec seuil optimal ---
ax1.hist(tous_pixels, bins=50, range=(0, 255), color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(x=seuil_otsu, color='red', linestyle='-', linewidth=3, label=f'Seuil Otsu = {seuil_otsu}')
ax1.fill_betweenx([0, ax1.get_ylim()[1] if ax1.get_ylim()[1] > 0 else 200], 
                   0, seuil_otsu, alpha=0.2, color='blue', label='Classe 0')
ax1.fill_betweenx([0, ax1.get_ylim()[1] if ax1.get_ylim()[1] > 0 else 200], 
                   seuil_otsu, 255, alpha=0.2, color='orange', label='Classe 1')
ax1.set_xlabel('Intensité des pixels', fontsize=11)
ax1.set_ylabel('Nombre de pixels', fontsize=11)
ax1.set_title('Histogramme de l\'image', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')
ax1.set_xlim(0, 255)

# Recalculer ylim après le plot
ax1.hist(tous_pixels, bins=50, range=(0, 255), color='steelblue', edgecolor='black', alpha=0.7)
ymax = ax1.get_ylim()[1]
ax1.axvline(x=seuil_otsu, color='red', linestyle='-', linewidth=3)
ax1.fill_betweenx([0, ymax], 0, seuil_otsu, alpha=0.2, color='blue')
ax1.fill_betweenx([0, ymax], seuil_otsu, 255, alpha=0.2, color='orange')

# --- 2. Courbe de variance intra-classe ---
ax2.plot(seuils, variances_intra, 'b-', linewidth=2, label='Variance intra-classe')
ax2.axvline(x=seuil_otsu, color='red', linestyle='--', linewidth=2)
ax2.scatter([seuil_otsu], [variance_min], color='red', s=150, zorder=5, 
            label=f'Minimum : T={seuil_otsu}')
ax2.set_xlabel('Seuil T', fontsize=11)
ax2.set_ylabel('Variance intra-classe (σ²_intra)', fontsize=11)
ax2.set_title('Variance intra-classe en fonction du seuil\n(Otsu minimise cette courbe)', 
              fontsize=12, fontweight='bold')
ax2.legend()
ax2.set_xlim(0, 255)
ax2.grid(True, alpha=0.3)

# Annotation
ax2.annotate(f'MINIMUM\nT = {seuil_otsu}\nσ²_intra = {variance_min:.0f}', 
             xy=(seuil_otsu, variance_min),
             xytext=(seuil_otsu + 40, variance_min + 500),
             fontsize=10,
             arrowprops=dict(arrowstyle='->', color='red'),
             bbox=dict(boxstyle='round', facecolor='lightyellow'))

# --- 3. Comparaison de 3 seuils ---
seuils_demo = [60, seuil_otsu, 200]
couleurs = ['green', 'red', 'purple']
titres = ['Trop bas', 'Optimal (Otsu)', 'Trop haut']

# Créer 3 sous-zones dans ax3
ax3.set_xlim(0, 300)
ax3.set_ylim(0, 100)
ax3.axis('off')
ax3.set_title('Comparaison : pourquoi le seuil Otsu est optimal ?', fontsize=12, fontweight='bold')

for idx, (seuil, couleur, titre) in enumerate(zip(seuils_demo, couleurs, titres)):
    # Position horizontale
    x_offset = idx * 100 + 10
    
    # Calculer les stats pour ce seuil
    c0 = tous_pixels[tous_pixels <= seuil]
    c1 = tous_pixels[tous_pixels > seuil]
    var_intra = calculer_variance_intra_classe(tous_pixels, seuil)
    
    # Mini histogramme (simulé avec du texte et des barres)
    ax_mini = fig.add_axes([0.1 + idx*0.3, 0.08, 0.25, 0.30])
    
    ax_mini.hist(c0, bins=30, range=(0, 255), alpha=0.7, color='blue', label='Classe 0')
    ax_mini.hist(c1, bins=30, range=(0, 255), alpha=0.7, color='orange', label='Classe 1')
    ax_mini.axvline(x=seuil, color=couleur, linestyle='-', linewidth=3)
    
    # Évaluation
    if seuil == seuil_otsu:
        evaluation = "✓ Classes homogènes"
        title_color = 'green'
    else:
        evaluation = "✗ Classes mélangées"
        title_color = 'red'
    
    ax_mini.set_title(f'{titre}\nT = {seuil}\n\nσ²_intra = {var_intra:.0f}\n{evaluation}', 
                      fontsize=10, color=title_color)
    ax_mini.set_xlabel('Intensité')
    ax_mini.set_xlim(0, 255)
    
    if idx == 0:
        ax_mini.set_ylabel('Nb pixels')
    
    ax_mini.tick_params(labelsize=8)

plt.tight_layout()
plt.savefig('otsu_courbe_variance.png', dpi=150, bbox_inches='tight')
plt.show()


#### Récap méthode d’Ōtsu 

1. Pour chaque seuil T possible (de 1 à 254) :
   - On sépare les pixels en 2 classes : C0 (≤T) et C1 (>T)
   - On calcule la variance DANS chaque classe
   - On fait la moyenne pondérée : $\sigma^2_{intra} = w_0 × \sigma^2_0 + w_1 × \sigma^2_1$

2. On trace la courbe $\sigma^2_{intra}$ en fonction de T

3. Le seuil optimal est celui qui MINIMISE cette variance
   → C'est le point le plus bas de la courbe

4. Interprétation :
   - Variance intra-classe FAIBLE = chaque classe est homogène
   - Cela signifie qu'on a bien séparé le fond des objets

Dans notre exemple :
- Seuil optimal : T = {seuil_otsu}
- Ce seuil se trouve dans la 'vallée' de l'histogramme
- Il sépare naturellement les deux pics (fond sombre / objets clairs)

In [ ]:
def seuil_otsu(img_gris):
    """Calcule le seuil optimal (Otsu)."""
    hist, _ = np.histogram(img_gris.flatten(), bins=256, range=(0, 256))
    hist_norm = hist / hist.sum()
    
    meilleur_seuil, meilleure_variance = 0, 0
    for t in range(1, 255):
        w0, w1 = hist_norm[:t+1].sum(), hist_norm[t+1:].sum()
        if w0 == 0 or w1 == 0: continue
        
        mu0 = (np.arange(t+1) * hist_norm[:t+1]).sum() / w0
        mu1 = (np.arange(t+1, 256) * hist_norm[t+1:]).sum() / w1
        variance_inter = w0 * w1 * (mu0 - mu1) ** 2
        
        if variance_inter > meilleure_variance:
            meilleure_variance = variance_inter
            meilleur_seuil = t
    return meilleur_seuil

seuil_opt = seuil_otsu(img_gris)
img_otsu = seuillage_simple(img_gris, seuil_opt)
print(f"Seuil Otsu : {seuil_opt}")

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].imshow(img_gris, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].hist(img_gris.flatten(), bins=256, color='gray', alpha=0.7)
axes[1].axvline(x=seuil_opt, color='red', linewidth=2, label=f'Otsu={seuil_opt}')
axes[1].legend()
axes[1].set_title('Histogramme')

axes[2].imshow(img_otsu, cmap='gray')
axes[2].set_title(f'Seuillage Otsu')
axes[2].axis('off')
plt.tight_layout()
plt.show()

Le seuillage d’Otsu permet d’isoler automatiquement le disque, qui est la  figure qui se détache plus nettement du fond que le rectangle gris foncé.

Enfin, ne perdons pas de vue non plus que si nous avons appliqué la méthode des seuils sur une image en niveau de gris (un canal = un seuil) – par exemple issue d’une combinaison des différents canaux, on peut bien sûr aussi définir un seuil en se focalisant sur un canal de couleur donné selon le traitement voulu. 

Voyons, pour l’empreinte digitale, quel est le seuil déterminé par cette méthode :

In [ ]:
# VOTRE CODE

# Déjà, voyons à quoi ressemble l’histogramme de cette image 

afficher_histogramme(fingerprint[:,:,0], titre="Image - emprunte digitale")

# calculons le seuil d’Otsu et effectuons le seuillage
seuil_emprunte = seuil_otsu(fingerprint)
fingerprint_otsu = seuillage_simple(fingerprint, seuil_emprunte)
print(f"Seuil Otsu pour l’emprunte : {seuil_emprunte}")
# Afficher
afficher_histogramme(fingerprint_otsu[:, :,0], titre="Traitement emprunte - Otsu")

On constate que le succès n’est pas vraiment au rendez-vous : en effet l’histogramme de l’image de départ n’est pas vraiment bimodal. Peut-être qu’en augmentant le contraste avant le seuillage le résultat serait meilleur ? Il faudrait éclaircir les zones claires / floues (avec une correction gamma par exemple) puis seuiller :

In [ ]:
fingerprint_egal = egaliser_histogramme(fingerprint)[0]
afficher_histogramme(fingerprint_egal[:,:,0], titre="Emprunte digitale - égalisation")

In [ ]:
fingerprint_corrige = correction_gamma(fingerprint_egal, 0.1)
afficher_histogramme(fingerprint_corrige[:,:,0], titre="Emprunte digitale - correction gamma")

In [ ]:
fingerprint_corrige = correction_gamma(fingerprint_corrige, 2)
afficher_histogramme(fingerprint_corrige[:,:,0], titre="Emprunte digitale - correction gamma")

Appliquons maintenant le seuillage d’Otsu :

In [ ]:
seuil_emprunte = seuil_otsu(fingerprint_corrige)
fingerprint_otsu = seuillage_simple(fingerprint_corrige, seuil_emprunte)
print(f"Seuil Otsu pour l’emprunte : {seuil_emprunte}")

afficher_histogramme(fingerprint_otsu[:, :,2], titre="Traitement emprunte - Otsu")

C’est beaucoup mieux, mais pas optimal. Il faudrait certainement appliquer une correction non linéaire qui offre un traitement plus différentiel dans les hautes/basses lumières que la correction gamma (une courbe en « S » par exemple) pour optimiser le contraste avant le seuillage. Ou utiliser des techniques plus sophistiqués qu’un simple seuillage, qui repèrerait directement les motifs dans les différentes zones de l’image. On voit dans cet exemple que le traitement de situations réelles est nettement plus compliqué que les exemples trop parfaits généralement vus en cours.

## Partie 4 : Convolution

### 4.1 : Filtres (ou masques)

#### 4.1.1 : la Convolution

La convolution est l'opération fondamentale du traitement d'images. Elle permet de transformer une image en combinant chaque pixel avec ses voisins selon une règle définie par un noyau (ou masque, ou filtre).
C’est une opération qui est également utilisé en *deep learning*, notamment dans des architectures dites *CNN* (convolution neural network), particulièrement performants dans la reconnaissance d’image. Ainsi bien comprendre cette notion vous aidera lorsque vous ferez du *deep learning*.

Cette méthode est capable de détecter des patterns particuliers dans une image (des changements de valeurs plus ou moins rapides, une organisation des valeurs dans une direction donnée, etc.), elle est donc très utile pour détecter des zones particulières d’une image, comme les contours des formes présentes dans l’image, etc.

##### Principe (intuition)

Comme dans tout ce que l’on a vu précédemment, on va chercher un moyen de calculer une nouvelle valeur (de sortie) pour un pixel donné dans une image, et ainsi créer une nouvelle image dont certaines caractéristiques auront reçu un traitement spécifique. Dans le cas de la convolution, au lieu de considérer uniquement ce pixel indépendemment des autres, on va prendre en compte tout son voisinage (par exemple une zone 3×3 – un « masque » – centrée sur lui), c’est à dire 3 pixels au dessus (le pixel immédiatement au dessus plus les deux diagonales), les trois en dessous (le pixel immédiatement au dessous plus les deux diagonales), et les deux pixels à ses côté (immédiatement à droite et à gauche). On va ensuite définir un noyau de convolution qui indique comment pondérer chaque voisin dans le calcul : si le masque est une matrice 3×3, le noyau correspondant est définit pas les valeurs des 9 coefficients de cette matrice.

Comment utilise-t-on ce noyau ?

Concrètement, on "pose" le noyau sur l'image, centré sur le pixel à transformer. On multiplie chaque valeur du noyau par le pixel correspondant de l'image, puis on fait la somme. Cette somme devient la nouvelle valeur du pixel.

Voici une image qui illustre le processus, qui apparaît plus compliqué quand on l’écrit qu’il n’est réellement :

![Schéma application masque convolution](./images/MasqueConvolution.png)

Vous comprenez pourquoi on parle aussi de « masque » de convolution. Alors que l’on considérait l’ensemble de l’image quand on faisait des manipulation à partir de l’histogramme, ici le filtrage est déterminé par des valeurs considérées localement (un sous ensemble de l’image), c’est pour ça qu’on parlera aussi de filtrage local.

**Note 1** : On constate que l’image transformée contient des valeurs qui ne font pas sens pour l’affichage (des valeurs négatives, etc.), il faut donc renormaliser ces valeurs, pour qu’elle soient comprises dans un intervalle $[0; 255]$. Cela est surtout vrai si on veut afficher l’image, on peut aussi destiner cette transformation à d’autres traitements.

**Note 2** : détail pratique, le masque faisant une certaine largeur ou hauteur, il faut gérer aussi l’approche des bords de l’image (le masque centré sur un pixel au bord « débordera » de l’image). On peut utiliser différentes stratégies en fonction de la situation, qui reposent généralement sur du *padding* : on fait comme s’il y avait des pixels tout autour de l’image. On peut fixer différentes valeurs pour ces pixels rajoutés :
* mise à zéro (valeur = 0)
* continuité (valeur = valeur du pixel le plus proche sur le bord)
* des choses plus sophistiquées : miroir (valeur du pixel symétrique par rapport au bord), sphérique/torique (valeur du pixel de l’autre côté, comme si l’image était plaquée sur un tore)

Exemples de noyaux classiques (le choix du noyau détermine l'effet produit) :
- Coefficients tous égaux (ex: 1/9 partout) -> Flou moyenneur (lisse l'image)
- Coefficients gaussiens (plus forts au centre) -> Flou gaussien (lisse en préservant mieux les structures)
- Coefficients positifs et négatifs -> Détection de contours (fait ressortir les transitions) ou amélioration de la netteté dans certains cas.
On trouve dans [l’article dédié sur Wikipedia](https://fr.wikipedia.org/wiki/Noyau_(traitement_d%27image)) un certains nombre d’exemples de noyaux avec une illustration de leurs effets.

##### Formulation mathématique
Maintenant que nous avons posé le principe, voyons comment nous pouvons le formuler du point de vue mathématique.

Pour une image $I$ et un noyau $K$, la convolution produit une nouvelle image $I’$ définie par :

$$
I’(x,y) = \sum_{i,j} K(i,j) \cdot I(x+i, y+j)
$$

où :

* $(x,y)$ est la position du pixel qu'on calcule dans l'image de sortie
* $(i,j)$ parcourt toutes les positions du noyau (par exemple de -1 à +1 pour un noyau 3×3)

$K(i,j)$ est le coefficient du noyau à la position $(i,j)$
$I(x+i,y+j)$ est l'intensité du pixel voisin correspondant dans l'image d'entrée

En d'autres termes : on fait une somme pondérée des pixels voisins, où les poids sont donnés par le noyau.

La formule ci-dessus peut sembler bien compliquée pour qui n’est pas habitué, mais en fait elle exprime une idée toute simple, illustrée par les schémas ci-dessous :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def demo_convolution_pas_a_pas():
    """
    Démonstration pas à pas du calcul de convolution sur un exemple simple.
    """
    
    # =========================================================================
    # Image et noyau simples pour l'exemple
    # =========================================================================
    
    # Image 6x6 avec des valeurs simples
    image = np.array([
        [10, 10, 10, 50, 50, 50],
        [10, 10, 10, 50, 50, 50],
        [10, 10, 10, 50, 50, 50],
        [10, 10, 10, 50, 50, 50],
        [10, 10, 10, 50, 50, 50],
        [10, 10, 10, 50, 50, 50]
    ], dtype=np.float64)
    
    # Noyau moyenneur 3x3
    noyau_moyenne = np.array([
        [1/9, 1/9, 1/9],
        [1/9, 1/9, 1/9],
        [1/9, 1/9, 1/9]
    ])
    
    # Noyau de détection de contours verticaux (Sobel X simplifié)
    noyau_contour = np.array([
        [-1, 0, 1],
        [-1, 0, 1],
        [-1, 0, 1]
    ])
    
    print("=" * 70)
    print("CONVOLUTION - EXEMPLE PAS À PAS")
    print("=" * 70)
    
    # =========================================================================
    # Affichage de l'image et du noyau
    # =========================================================================
    
    print("\n" + "─" * 70)
    print("IMAGE D'ENTRÉE (6×6)")
    print("─" * 70)
    print("\nL'image contient une transition verticale (10 → 50) :")
    print()
    for row in image:
        print("  " + "  ".join([f"{int(v):3d}" for v in row]))
    
    print("\n" + "─" * 70)
    print("NOYAU MOYENNEUR (3×3)")
    print("─" * 70)
    print()
    for row in noyau_moyenne:
        print("  " + "  ".join([f"{v:.3f}" for v in row]))
    print("\n→ Chaque coefficient = 1/9 ≈ 0.111")
    print("→ La somme des coefficients = 1 (préserve la luminosité moyenne)")
    
    # =========================================================================
    # Calcul détaillé pour UN pixel
    # =========================================================================
    
    print("\n" + "─" * 70)
    print("CALCUL DÉTAILLÉ POUR LE PIXEL (2, 3)")
    print("─" * 70)
    
    x, y = 2, 3  # Position du pixel à calculer
    
    print(f"\nOn veut calculer G({x}, {y}) — le pixel à la ligne {x}, colonne {y}")
    print("\nÉtape 1 : Extraire le voisinage 3×3 centré sur ({}, {})".format(x, y))
    print()
    
    # Extraire le voisinage
    voisinage = image[x-1:x+2, y-1:y+2]
    
    print("Voisinage extrait :")
    print("┌─────┬─────┬─────┐")
    for i, row in enumerate(voisinage):
        print("│", end="")
        for j, v in enumerate(row):
            print(f" {int(v):3d} │", end="")
        print()
        if i < 2:
            print("├─────┼─────┼─────┤")
    print("└─────┴─────┴─────┘")
    
    print("\nÉtape 2 : Multiplier élément par élément avec le noyau")
    print()
    print("  Voisinage       ×       Noyau         =     Produits")
    print()
    
    produits = voisinage * noyau_moyenne
    
    for i in range(3):
        v_str = "  ".join([f"{int(voisinage[i,j]):3d}" for j in range(3)])
        k_str = "  ".join([f"{noyau_moyenne[i,j]:.3f}" for j in range(3)])
        p_str = "  ".join([f"{produits[i,j]:5.2f}" for j in range(3)])
        
        if i == 1:
            print(f"  [{v_str}]   ×   [{k_str}]   =   [{p_str}]")
        else:
            print(f"  [{v_str}]       [{k_str}]       [{p_str}]")
    
    print("\nÉtape 3 : Sommer tous les produits")
    print()
    print("  G({}, {}) = ".format(x, y), end="")
    termes = []
    for i in range(3):
        for j in range(3):
            termes.append(f"{produits[i,j]:.2f}")
    print(" + ".join(termes[:3]))
    print("           + " + " + ".join(termes[3:6]))
    print("           + " + " + ".join(termes[6:9]))
    print()
    resultat = produits.sum()
    print(f"  G({x}, {y}) = {resultat:.2f}")
    print()
    print(f"→ Le pixel (2,3) qui valait {int(image[x,y])} devient {resultat:.1f}")
    print("  (moyenne des 9 voisins : 6 pixels à 10 et 3 pixels à 50)")
    
    # =========================================================================
    # Calcul pour un pixel sur le contour
    # =========================================================================
    
    print("\n" + "─" * 70)
    print("CALCUL POUR LE PIXEL (2, 2) — au bord de la transition")
    print("─" * 70)
    
    x2, y2 = 2, 2
    voisinage2 = image[x2-1:x2+2, y2-1:y2+2]
    produits2 = voisinage2 * noyau_moyenne
    resultat2 = produits2.sum()
    
    print(f"\nVoisinage de ({x2}, {y2}) :")
    for row in voisinage2:
        print("  " + "  ".join([f"{int(v):3d}" for v in row]))
    
    print(f"\nG({x2}, {y2}) = {resultat2:.2f}")
    print(f"→ Le pixel (2,2) qui valait {int(image[x2,y2])} devient {resultat2:.1f}")
    print("  (9 pixels à 10 → moyenne = 10)")
    
    # =========================================================================
    # Visualisation graphique
    # =========================================================================
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 9))
    
    # --- Ligne 1 : Convolution avec noyau moyenneur ---
    
    # Image originale avec zone surlignée
    ax1 = axes[0, 0]
    im1 = ax1.imshow(image, cmap='gray', vmin=0, vmax=60)
    ax1.set_title('Image originale\n(transition 10 → 50)', fontweight='bold')
    # Annoter les valeurs
    for i in range(6):
        for j in range(6):
            ax1.text(j, i, f'{int(image[i,j])}', ha='center', va='center',
                    fontsize=10, color='red' if image[i,j] < 30 else 'blue')
    # Rectangle pour le voisinage
    rect = patches.Rectangle((y-1.5, x-1.5), 3, 3, linewidth=3,
                               edgecolor='lime', facecolor='none')
    ax1.add_patch(rect)
    ax1.set_xticks(range(6))
    ax1.set_yticks(range(6))
    ax1.set_xlabel('Colonne (y)')
    ax1.set_ylabel('Ligne (x)')
    
    # Noyau
    ax2 = axes[0, 1]
    ax2.imshow(noyau_moyenne, cmap='Blues', vmin=0, vmax=0.2)
    ax2.set_title('Noyau moyenneur\n(3×3, somme=1)', fontweight='bold')
    for i in range(3):
        for j in range(3):
            ax2.text(j, i, f'{noyau_moyenne[i,j]:.2f}', ha='center', va='center',
                    fontsize=12, color='black')
    ax2.set_xticks([0, 1, 2])
    ax2.set_yticks([0, 1, 2])
    ax2.set_xticklabels(['-1', '0', '+1'])
    ax2.set_yticklabels(['-1', '0', '+1'])
    
    # Produits
    ax3 = axes[0, 2]
    ax3.imshow(produits, cmap='Oranges', vmin=0, vmax=6)
    ax3.set_title(f'Produits (voisinage × noyau)\nSomme = {resultat:.1f}', fontweight='bold')
    for i in range(3):
        for j in range(3):
            ax3.text(j, i, f'{produits[i,j]:.2f}', ha='center', va='center',
                    fontsize=11, color='black')
    ax3.set_xticks([0, 1, 2])
    ax3.set_yticks([0, 1, 2])
    
    # Résultat complet
    def convolution_complete(img, noyau):
        h, w = img.shape
        kh, kw = noyau.shape
        pad = kh // 2
        img_pad = np.pad(img, pad, mode='reflect')
        resultat = np.zeros_like(img)
        for i in range(h):
            for j in range(w):
                resultat[i, j] = np.sum(img_pad[i:i+kh, j:j+kw] * noyau)
        return resultat
    
    img_moyenne = convolution_complete(image, noyau_moyenne)
    
    ax4 = axes[0, 3]
    ax4.imshow(img_moyenne, cmap='gray', vmin=0, vmax=60)
    ax4.set_title('Résultat : image floutée\n(transition adoucie)', fontweight='bold')
    for i in range(6):
        for j in range(6):
            ax4.text(j, i, f'{img_moyenne[i,j]:.0f}', ha='center', va='center',
                    fontsize=9, color='red' if img_moyenne[i,j] < 30 else 'blue')
    ax4.set_xticks(range(6))
    ax4.set_yticks(range(6))
    
    # --- Ligne 2 : Convolution avec noyau de contours ---
    
    ax5 = axes[1, 0]
    ax5.imshow(image, cmap='gray', vmin=0, vmax=60)
    ax5.set_title('Image originale', fontweight='bold')
    for i in range(6):
        for j in range(6):
            ax5.text(j, i, f'{int(image[i,j])}', ha='center', va='center',
                    fontsize=10, color='red' if image[i,j] < 30 else 'blue')
    ax5.set_xticks(range(6))
    ax5.set_yticks(range(6))
    
    ax6 = axes[1, 1]
    cmap_contour = plt.cm.RdBu_r
    ax6.imshow(noyau_contour, cmap=cmap_contour, vmin=-1.5, vmax=1.5)
    ax6.set_title('Noyau détection contours\n(dérivée horizontale)', fontweight='bold')
    for i in range(3):
        for j in range(3):
            val = noyau_contour[i, j]
            color = 'white' if abs(val) > 0.5 else 'black'
            ax6.text(j, i, f'{int(val):+d}', ha='center', va='center',
                    fontsize=14, fontweight='bold', color=color)
    ax6.set_xticks([0, 1, 2])
    ax6.set_yticks([0, 1, 2])
    ax6.set_xticklabels(['-1', '0', '+1'])
    ax6.set_yticklabels(['-1', '0', '+1'])
    
    # Calcul détaillé pour contours
    voisinage_contour = image[2:5, 2:5]
    produits_contour = voisinage_contour * noyau_contour
    
    ax7 = axes[1, 2]
    ax7.imshow(produits_contour, cmap=cmap_contour, vmin=-60, vmax=60)
    ax7.set_title(f'Produits\nSomme = {produits_contour.sum():.0f}', fontweight='bold')
    for i in range(3):
        for j in range(3):
            val = produits_contour[i, j]
            color = 'white' if abs(val) > 20 else 'black'
            ax7.text(j, i, f'{val:.0f}', ha='center', va='center',
                    fontsize=11, color=color)
    ax7.set_xticks([0, 1, 2])
    ax7.set_yticks([0, 1, 2])
    
    img_contour = convolution_complete(image, noyau_contour)
    
    ax8 = axes[1, 3]
    im8 = ax8.imshow(img_contour, cmap=cmap_contour, vmin=-150, vmax=150)
    ax8.set_title('Résultat : contours détectés\n(fort gradient à la transition)', fontweight='bold')
    for i in range(6):
        for j in range(6):
            val = img_contour[i, j]
            color = 'white' if abs(val) > 50 else 'black'
            ax8.text(j, i, f'{val:.0f}', ha='center', va='center',
                    fontsize=9, color=color)
    ax8.set_xticks(range(6))
    ax8.set_yticks(range(6))
    
    plt.suptitle('Convolution pas à pas : deux noyaux, deux effets', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

demo_convolution_pas_a_pas()

##### Interprétation :

* NOYAU MOYENNEUR (flou)
    * Tous les coefficients sont positifs et égaux (1/9)
    * Chaque pixel devient la moyenne de ses voisins
    * Effet : la transition brutale 10→50 devient progressive
    * Les colonnes 2 et 3 ont des valeurs intermédiaires (~23 et ~37)
    
* NOYAU DÉTECTION DE CONTOURS (dérivée)
    * Coefficients négatifs à gauche (-1), nuls au centre (0), positifs à droite (+1)
    * Calcul : (voisins droite) - (voisins gauche)
    * Si pas de variation horizontale → résultat proche de 0
    * Si transition gauche→droite → résultat positif (contour détecté!)
    
    * Colonnes 0,1 : que des 10 autour → 10×(1+1+1) + 10×(-1-1-1) = 0
    * Colonne 2,3 : transition → 50×(1+1+1) + 10×(-1-1-1) = 150 - 30 = 120
    * Colonnes 4,5 : que des 50 autour → 50×(1+1+1) + 50×(-1-1-1) = 0

Ainsi on peut créer des filtres qui sont sensibles à certaines variations selon certaines directions, ce qui nous permettrait de détecter des contours :

* détection horizontale avec le noyau :
  $$
  K = \begin{bmatrix} -1 & 0 & +1 \\ -1 & 0 & +1 \\ -1 & 0 & +1 \end{bmatrix}
  $$
* détection horizontale :
  $$
  K = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ +1 & +1 & +1 \end{bmatrix}
  $$
* en diagonale (+45° ou -45°) :
  $$
  K_{+45°} = \begin{bmatrix} 0 & 0 & 0 \\ 0 & -1 & 0 \\ 0 & 0 & +1 \end{bmatrix}
  \qquad
  K_{-45°} = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & +1 \\ 0 & -1 & 0 \end{bmatrix}
  $$

Bien sûr, détecter des variations selon des directions privilégiées ont d’autres applications que détecter des contours, la détection de ces motifs de bases jouent aussi leur rôle dans la reconnaissance de forme, et sont au cœur de l’architecture CNN en *deep learning*.

On peut créer des noyaux plus élaborés en choisissant des valeurs différentes de 1 pour certains coéfficient, ce qui permet de pondérer certaines directions (pixels voisins). Nous ne sommes pas non plus limité à des matices 3×3, par exemple on peut aussi utiliser des matrices 5×5, qui prendront alors en compte des pixels plus éloignés (masques plus grands).

Nous allons voir un filtre très populaire en détection de contour : le filtre de Sobel.

#### 4.1.2 : filtre de Sobel – Détection de contours

Le filtre de Sobel est l'outil classique pour détecter les contours dans une image. Un contour correspond à une zone où l'intensité des pixels change brusquement — autrement dit, là où le gradient (la pente) de l'image est fort.

##### Principe
Pour détecter un contour, on cherche à mesurer à quelle vitesse l'intensité change quand on se déplace dans l'image. Cette variation s'appelle le gradient. Comme l'image est en 2D, on calcule le gradient selon deux directions :

- Gradient horizontal $G_x$ : détecte les contours verticaux (transitions gauche-droite)
- Gradient vertical $G_y$ : détecte les contours horizontaux (transitions haut-bas)

Pour effectuer ces détection, Sobel propose deux noyaux (des matrices 3×3) définis ainsi :
$$
K_x = \begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix}
\qquad
K_y = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix}
$$

##### Compréhension intuitive

Regardons $K_x$ de plus près :
- Colonne de gauche : coefficients négatifs → on soustrait les voisins de gauche
- Colonne de droite : coefficients positifs → on ajoute les voisins de droite
- Le coefficient central (±2) donne plus de poids au voisin direct

Le résultat $G_x$ sera donc élevé s'il y a une forte différence entre la droite et la gauche, c'est-à-dire un contour vertical. Le noyau $K_y$ fonctionne de la même façon mais verticalement.

Un autre indice important est la **magnitude des gradients calculés** : une fois $G_x$ et $G_y$ calculés, on combine les deux pour obtenir la force totale du contour en chaque pixel.
$$
G = \sqrt{G_x^2 + G_y^2}
$$
Cette magnitude est élevée là où il y a un contour (quelle que soit son orientation) et proche de zéro dans les zones uniformes. On obtiendra donc une image de sortie où les pixels clairs (de valeurs élevés) correspondront aux contours détectés, les autres pixels seront sombres.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def demo_filtre_sobel():
    """
    Démonstration du filtre de Sobel pour la détection de contours.
    """
    
    # =========================================================================
    # Création d'une image test avec différents types de contours
    # =========================================================================
    
    np.random.seed(42)
    
    h, w = 200, 250
    img = np.ones((h, w), dtype=np.float64) * 180
    
    # Rectangle (contours horizontaux et verticaux)
    img[30:80, 30:100] = 60
    
    # Cercle (contours dans toutes les directions)
    centre_x, centre_y = 180, 60
    for i in range(h):
        for j in range(w):
            if (i - centre_y)**2 + (j - centre_x)**2 < 35**2:
                img[i, j] = 50
    
    # Triangle (contours diagonaux)
    for i in range(50):
        gauche = 60 + i
        droite = 140 - i
        if gauche < droite:
            img[100 + i, gauche:droite] = 70
    
    # Dégradé progressif (pas de contour net)
    for j in range(50, 100):
        img[160:190, j] = 80 + (j - 50) * 2
    
    # Léger bruit
    img += np.random.normal(0, 3, (h, w))
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    # =========================================================================
    # Définition des noyaux de Sobel
    # =========================================================================
    
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float64)
    
    sobel_y = np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float64)
    
    # =========================================================================
    # Fonction de convolution
    # =========================================================================
    
    def convoluer(image, noyau):
        """Applique une convolution avec padding par réflexion."""
        h, w = image.shape
        kh, kw = noyau.shape
        pad = kh // 2
        
        img_pad = np.pad(image.astype(np.float64), pad, mode='reflect')
        resultat = np.zeros((h, w), dtype=np.float64)
        
        for i in range(h):
            for j in range(w):
                resultat[i, j] = np.sum(img_pad[i:i+kh, j:j+kw] * noyau)
        
        return resultat
    
    # =========================================================================
    # Application des filtres de Sobel
    # =========================================================================
    
    # Gradients directionnels
    Gx = convoluer(img, sobel_x)
    Gy = convoluer(img, sobel_y)
    
    # Magnitude du gradient
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    # Normalisation pour affichage
    magnitude_norm = (magnitude / magnitude.max() * 255).astype(np.uint8)
    
    # =========================================================================
    # Visualisation principale
    # =========================================================================
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # --- Ligne 1 : Image et noyaux ---
    
    # Image originale
    axes[0, 0].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[0, 0].set_title('Image originale', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    # Noyau Sobel X
    ax_kx = axes[0, 1]
    im_kx = ax_kx.imshow(sobel_x, cmap='RdBu_r', vmin=-2, vmax=2)
    ax_kx.set_title('Noyau Sobel X\n(détecte contours verticaux)', fontsize=11, fontweight='bold')
    for i in range(3):
        for j in range(3):
            val = sobel_x[i, j]
            color = 'white' if abs(val) > 1 else 'black'
            ax_kx.text(j, i, f'{int(val):+d}', ha='center', va='center',
                      fontsize=16, fontweight='bold', color=color)
    ax_kx.set_xticks([0, 1, 2])
    ax_kx.set_yticks([0, 1, 2])
    ax_kx.set_xticklabels(['G', 'C', 'D'])
    ax_kx.set_yticklabels(['H', 'C', 'B'])
    
    # Noyau Sobel Y
    ax_ky = axes[0, 2]
    im_ky = ax_ky.imshow(sobel_y, cmap='RdBu_r', vmin=-2, vmax=2)
    ax_ky.set_title('Noyau Sobel Y\n(détecte contours horizontaux)', fontsize=11, fontweight='bold')
    for i in range(3):
        for j in range(3):
            val = sobel_y[i, j]
            color = 'white' if abs(val) > 1 else 'black'
            ax_ky.text(j, i, f'{int(val):+d}', ha='center', va='center',
                      fontsize=16, fontweight='bold', color=color)
    ax_ky.set_xticks([0, 1, 2])
    ax_ky.set_yticks([0, 1, 2])
    ax_ky.set_xticklabels(['G', 'C', 'D'])
    ax_ky.set_yticklabels(['H', 'C', 'B'])
    
    # --- Ligne 2 : Résultats ---
    
    # Gradient X
    ax_gx = axes[1, 0]
    vmax_g = max(abs(Gx.min()), abs(Gx.max()))
    im_gx = ax_gx.imshow(Gx, cmap='RdBu_r', vmin=-vmax_g, vmax=vmax_g)
    ax_gx.set_title('Gradient Gx\n(positif=transition →, négatif=transition ←)', 
                    fontsize=11, fontweight='bold')
    ax_gx.axis('off')
    plt.colorbar(im_gx, ax=ax_gx, fraction=0.046, pad=0.04)
    
    # Gradient Y
    ax_gy = axes[1, 1]
    vmax_g = max(abs(Gy.min()), abs(Gy.max()))
    im_gy = ax_gy.imshow(Gy, cmap='RdBu_r', vmin=-vmax_g, vmax=vmax_g)
    ax_gy.set_title('Gradient Gy\n(positif=transition ↓, négatif=transition ↑)', 
                    fontsize=11, fontweight='bold')
    ax_gy.axis('off')
    plt.colorbar(im_gy, ax=ax_gy, fraction=0.046, pad=0.04)
    
    # Magnitude
    ax_mag = axes[1, 2]
    im_mag = ax_mag.imshow(magnitude_norm, cmap='hot')
    ax_mag.set_title('Magnitude |G| = √(Gx² + Gy²)\n(intensité du contour)', 
                     fontsize=11, fontweight='bold')
    ax_mag.axis('off')
    plt.colorbar(im_mag, ax=ax_mag, fraction=0.046, pad=0.04)
    
    plt.suptitle('Filtre de Sobel : détection de contours', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # =========================================================================
    # Zoom sur une zone pour expliquer les signes
    # =========================================================================
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # Zone du rectangle (contient des contours verticaux et horizontaux nets)
    y1, y2 = 20, 90
    x1, x2 = 20, 110
    
    axes[0].imshow(img[y1:y2, x1:x2], cmap='gray', vmin=0, vmax=255)
    axes[0].set_title('Zoom : rectangle\n(contours nets)', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(Gx[y1:y2, x1:x2], cmap='RdBu_r', vmin=-400, vmax=400)
    axes[1].set_title('Gx : bords gauche (-) et droit (+)\ndu rectangle détectés', fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(Gy[y1:y2, x1:x2], cmap='RdBu_r', vmin=-400, vmax=400)
    axes[2].set_title('Gy : bords haut (-) et bas (+)\ndu rectangle détectés', fontweight='bold')
    axes[2].axis('off')
    
    axes[3].imshow(magnitude[y1:y2, x1:x2], cmap='hot')
    axes[3].set_title('Magnitude : tous les\ncontours combinés', fontweight='bold')
    axes[3].axis('off')
    
    plt.suptitle('Interprétation des signes du gradient', fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.show()

demo_filtre_sobel()

##### Fonctionnement du filtre de Sobel

* GRADIENT Gx (noyau horizontal)
    * Gx > 0 (rouge) : l'intensité AUGMENTE de gauche à droite
                       → contour vertical, côté DROIT d'un objet sombre
    * Gx < 0 (bleu)  : l'intensité DIMINUE de gauche à droite
                       → contour vertical, côté GAUCHE d'un objet sombre
    * Gx ≈ 0         : pas de variation horizontale (zone uniforme)
    
* GRADIENT Gy (noyau vertical)
    * Gy > 0 (rouge) : l'intensité AUGMENTE de haut en bas
                       → contour horizontal, côté BAS d'un objet sombre
    * Gy < 0 (bleu)  : l'intensité DIMINUE de haut en bas
                       → contour horizontal, côté HAUT d'un objet sombre
    * Gy ≈ 0         : pas de variation verticale (zone uniforme)
    
* MAGNITUDE |G|
    * Combine Gx et Gy : $|G| = \sqrt{G_x^2 + G_y^2)}$
    * Élevée sur TOUS les contours (quelle que soit l'orientation)
    * Proche de 0 dans les zones uniformes
    * Le dégradé progressif (en bas) produit une magnitude faible
      car la transition est lente, pas un contour net
    
* Notes :
    * Le cercle montre des contours dans toutes les directions
      → Gx et Gy sont complémentaires
    * Le triangle montre des contours diagonaux
      → Gx ET Gy sont non-nuls sur les bords obliques

In [ ]:
def convolution_2d(img, noyau):
    """Convolution 2D."""
    h, w = img.shape
    kh, kw = noyau.shape
    pad_h, pad_w = kh // 2, kw // 2
    
    img_padded = np.pad(img.astype(float), ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
    resultat = np.zeros((h, w), dtype=float)
    
    for i in range(h):
        for j in range(w):
            resultat[i, j] = np.sum(img_padded[i:i+kh, j:j+kw] * noyau)
    return resultat

# Filtres de Sobel
sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])

gx = convolution_2d(img_gris, sobel_x)
gy = convolution_2d(img_gris, sobel_y)
magnitude = np.sqrt(gx**2 + gy**2)
magnitude = (magnitude / magnitude.max() * 255).astype(np.uint8)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes[0,0].imshow(img_gris, cmap='gray'); axes[0,0].set_title('Original'); axes[0,0].axis('off')
axes[0,1].imshow(np.abs(gx), cmap='gray'); axes[0,1].set_title('Sobel X'); axes[0,1].axis('off')
axes[1,0].imshow(np.abs(gy), cmap='gray'); axes[1,0].set_title('Sobel Y'); axes[1,0].axis('off')
axes[1,1].imshow(magnitude, cmap='gray'); axes[1,1].set_title('Magnitude'); axes[1,1].axis('off')
plt.tight_layout()
plt.show()

Magique !

#### 4.1.3 Autres noyaux de convolution notables

##### Filtres de lissage (réduction du bruit)

###### Filtre moyenneur (box blur)
$$
K = \frac{1}{9}\begin{bmatrix} 1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix}
$$
Remplace chaque pixel par la moyenne de ses voisins. Simple mais crée du flou sur les contours.

###### Filtre gaussien
$$
K = \frac{1}{16}\begin{bmatrix} 1 & 2 & 1 \\ 2 & 4 & 2 \\ 1 & 2 & 1 \end{bmatrix}
$$
Moyenne pondérée où les voisins immédiats comptent plus. Flou plus naturel, préserve mieux les structures que le moyenneur.

###### Filtre médian

Calculer la médiane des valeurs des pixels dans le masque, remplacer la valeur du pixel central par cette médiane.
Cette méthode donne un rendu moins flou que les noyaux précédents, par contre elle a tendance à détruire les angles. Particulièrement efficace contre le bruit type « neige » ou « poivre et sel ».

##### Filtres de détection de contours

###### Filtre de Prewitt
$$
K_x = \begin{bmatrix} -1 & 0 & +1 \\ -1 & 0 & +1 \\ -1 & 0 & +1 \end{bmatrix}
\qquad
K_y = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ +1 & +1 & +1 \end{bmatrix}
$$

Similaire à Sobel mais sans pondération centrale. Moins sensible au bruit que Sobel.

###### Filtre laplacien
$$
K_4 = \begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix}
\quad \text{ou} \quad
K_8 = \begin{bmatrix} 1 & 1 & 1 \\ 1 & -8 & 1 \\ 1 & 1 & 1 \end{bmatrix}
$$

On peut aussi inverser les signes (positif pour le coefficient central, négatif sur les coefficients en périphérie). Le Laplacien est un opérateur très impoortant en physique et en traitement du signal, notamment pour filtrer. Il est aussi utilisé pour une catégorie de réseau de neurones (cartes auto-organisatrices notamment, et des modèles dits « *winner take all* »). Il repose sur le calcul d’une dérivée seconde :
$$
\nabla^2 = \frac{\partial^2 }{\partial x^2}+\frac{\partial^2 }{\partial y^2}
$$

> **Explications optionnelles** : *si ça fait déjà trop de math pour vous, vous pouvez sauter cette partie et y revenir plus tard*
> 
> Quel est le rapport avec les matrices ci-dessus ? Comment à partir d’une dérivée seconde arrive-t-on à ces valeurs précises ?
>
> Calculer une dérivée sur la position des pixels dans un écran ne peut se faire que par approximation : il s’agit de valeurs discrètes (et non continues).
> En réalité, l’approximation va consister en le calcul d’une simple différence entre des pixels voisins (p. ex. avec le pixel à droite pour une dérivation selon l’axe x, direction horizontale) :
> $$ \frac{\partial f }{\partial x}≈f(x+1)−f(x) $$
> Calculer une dérivée seconde, c’est dériver deux fois. C’est à dire qu’on va cacluler pour tous les pixels la dérivé première (différence de tous les pixels avec leurs voisins de droite), et ensuite, pour deuxième dérivée, calculer de la même manière la différence entre ces différences (on  va faire deux soustractions imbriquées) :
> $$\frac{\partial^2 f }{\partial x^2} = \frac{\partial }{\partial x}(\frac{\partial f }{\partial x}) ≈ [f(x+1)−f(x)]−[f(x)−f(x−1)]$$
> $$= f(x+1)−2f(x)+f(x−1)$$
> On constate donc que finalement cela revient à faire une somme entre le pixel central affecté d’un coefficient -2 et les pixels immédiatement à gauche et à droite affectés pour leur part d’un coefficient +1.
> C’est exactement les mêmes calculs pour la direction horizontale, donc une somme pour laquelle le pixel central aura un coefficient -2 et les pixel immédiatement au dessus ou en dessous un coefficient +1
> Si on « assemble » ces opération, on voit que cela correspond à un noyau 3×3 avec un coefficent (-2) + (-2) pour le pixel central, et +1 pour les pixels dans les 4 directions haut, bas, gauche, droite, soit la matrice :
> $$\begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix}$$
> On comprend de la même manière que le noyau
> $$\begin{bmatrix} 1 & 1 & 1 \\ 1 & -8 & 1 \\ 1 & 1 & 1 \end{bmatrix}$$
> correspond à une dérivée seconde dans les huits directions (avec les diagonales en plus), et dans ce cas comme on ajoute deux directions par rapport à la situation précédente, le pixel central reçoit 4 fois un coefficient -2 soit -8 au total.

$K_8$ détecte les contours dans toutes les directions en une seule opération. Les noyaux laplaciens sont néanmoins très sensibles au bruit.

Pour mieux appréhender comment fonctionne cette transformation, voici un bout de code qui permet de visualiser un Laplacien qu’on va appliquer à une Gaussienne qu’on va assimiler à un pic de luminosité sur l’image (zone avec des pixels lumineux au centre et qui s’atténuent assez vite, comme sur un contour). On fait une représentation 3D : x et y sont les positions des pixels sur l’image, alors que z va représenter la  valeur (luminosité) d’un pixel de coordonnée (x, y).

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
#pio.renderers.default = "iframe" # définir ce renderer si problème d’affichage avec px
# alternative à tester :  installer !pip install jupyterlab-plotly-extension

# Grille 2D
x = np.linspace(-3, 3, 80)
y = np.linspace(-3, 3, 80)
X, Y = np.meshgrid(x, y)

# Fonction : une gaussienne (bosse)
Z = np.exp(-(X**2 + Y**2))

# Laplacien analytique
laplacien = (4*X**2 + 4*Y**2 - 4) * np.exp(-(X**2 + Y**2))

In [ ]:
# Figure 1 : La fonction en 3D
fig1 = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Viridis')])
fig1.update_layout(title="Fonction f(x,y) = exp(-(x² + y²))")
fig1.show()

In [ ]:
# Figure 2 : Son Laplacien en 3D
fig2 = go.Figure(data=[go.Surface(x=X, y=Y, z=laplacien, colorscale='RdBu_r', cmid=0)])
fig2.update_layout(title="Laplacien ∇²f — négatif (bleu) au sommet de la bosse")
fig2.show()

On voit que pour une fonction Gaussienne (qui représente une bosse ou un pic de luminosité = maximum au centre) le Laplacien devient fortement **négatif** au centre (en bleu sur la figure), et **positif** sur les bords (en rouge sur la figure). Parfois on trouve l’inverse si on calcule l’opposé du Laplacien, selon les usages qu’on veut en faire. On parle aussi de fonction en forme de « chapeau mexicain ».

Ce qu’il est important de retenir, c’est que le pixel (ou groupe de pixels) central est détecté car sa transformation est en valeur absolue nettement plus haute que celle de ses voisins, qui eux seront affecté d’un signe inverse avec une valeur absolue nettement plus petite. Le Laplacien amplifie donc (détecte) le pic de luminosité, et inhibe les voisins avec une luminosité plus faible. C’est pour cela que l’on parle de *winner take all* : non seulement le pixel le plus lumineux est sélectionné, mais les piels voisins concurrents, un peu moins lumineux, sont carrément réduits encore plus que des pixels encore moins lumineux mais plus lointains. 

###### Filtre de Roberts
$$
K_x = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}
\qquad
K_y = \begin{bmatrix} 0 & 1 \\ -1 & 0 \end{bmatrix}
$$
Noyau 2×2 pour détecter les contours diagonaux. Rapide mais très sensible au bruit.

##### Filtres de netteté (sharpening)
###### Filtre d'accentuation (sharpen)
$$
K = \begin{bmatrix} 0 & -1 & 0 \\ -1 & 5 & -1 \\ 0 & -1 & 0 \end{bmatrix}
$$
Renforce les contours en amplifiant les différences avec les voisins. Le coefficient central > somme des autres.

###### Filtre unsharp mask (masque flou)
Principe : Image nette = Image originale + k × (Image originale − Image floue)
Accentue les détails en ajoutant la différence entre l'image et sa version floutée.

##### Filtre de détection de motifs

###### Filtre de relief (emboss)
$$
K = \begin{bmatrix} -2 & -1 & 0 \\ -1 & 1 & 1 \\ 0 & 1 & 2 \end{bmatrix}
$$
Crée un effet de relief 3D en accentuant les transitions dans une direction diagonale.


##### Récapitulatif :
| Noyau     | Taille      | Application           | Somme des coefficients | 
|-----------|-------------|-----------------------|------------------------|
| Moyenneur | 3×3 ou plus | Flou, réduction bruit | 1                      |
| Gaussien  | 3×3 ou plus | Flou naturel          | 1                      |
| Sobel     | 3×3 | Gradient, contours            | 0                      | 
| Prewitt   | 3×3 | Gradient, contours            | 0                      | 
| Laplacien | 3×3 | Contours toutes directions    | 0                      |
| Sharpen   | 3×3 | Accentuation netteté          | 1                      | 
| Emboss    | 3×3 | Effet relief                  | ~1                     |

Règle pratique :

* Somme = 1 → préserve la luminosité moyenne (flou, netteté)
* Somme = 0 → extrait les variations (contours, gradients)

#### 4.1.4 Exercice (optionnel – à faire chez vous)

À l’image de ce que nous avons fait pour le filtre de Sobel plus haut, écrivez un bout de code qui permet de visualiser les effets et le fonctionnement des différents noyaux vu ci-dessus (vous pouvez aussi expérimenter pour créer vos propres noyaux).
Vous pouvez aussi tester sur des images / photo… et créer votre propre appli de traitement d’image :) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def demo_tous_les_noyaux():
    """
    Démonstration visuelle de différents noyaux de convolution.
    """
    
    # =========================================================================
    # Création d'une image test riche en détails
    # =========================================================================
    
    np.random.seed(42)
    
    h, w = 180, 220
    img = np.ones((h, w), dtype=np.float64) * 160
    
    # Rectangle
    img[20:70, 20:80] = 60
    
    # Cercle
    for i in range(h):
        for j in range(w):
            if (i - 45)**2 + (j - 140)**2 < 30**2:
                img[i, j] = 50
    
    # Dégradé
    for j in range(20, 80):
        img[90:130, j] = 60 + (j - 20) * 2.5
    
    # Lignes fines (pour tester la netteté)
    img[140:145, 20:200] = 40
    img[150:152, 20:200] = 40
    img[156:157, 20:200] = 40
    
    # Texture (damier)
    for i in range(90, 130):
        for j in range(120, 160):
            if (i + j) % 10 < 5:
                img[i, j] = 80
    
    # Bruit léger
    img += np.random.normal(0, 5, (h, w))
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    # =========================================================================
    # Définition de tous les noyaux
    # =========================================================================
    
    noyaux = {
        'Identité': np.array([
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ], dtype=np.float64),
        
        'Moyenneur\n(box blur)': np.array([
            [1, 1, 1],
            [1, 1, 1],
            [1, 1, 1]
        ], dtype=np.float64) / 9,
        
        'Gaussien\n3×3': np.array([
            [1, 2, 1],
            [2, 4, 2],
            [1, 2, 1]
        ], dtype=np.float64) / 16,
        
        'Gaussien\n5×5': np.array([
            [1,  4,  6,  4, 1],
            [4, 16, 24, 16, 4],
            [6, 24, 36, 24, 6],
            [4, 16, 24, 16, 4],
            [1,  4,  6,  4, 1]
        ], dtype=np.float64) / 256,
        
        'Sobel X\n(contours V)': np.array([
            [-1, 0, 1],
            [-2, 0, 2],
            [-1, 0, 1]
        ], dtype=np.float64),
        
        'Sobel Y\n(contours H)': np.array([
            [-1, -2, -1],
            [ 0,  0,  0],
            [ 1,  2,  1]
        ], dtype=np.float64),
        
        'Prewitt X': np.array([
            [-1, 0, 1],
            [-1, 0, 1],
            [-1, 0, 1]
        ], dtype=np.float64),
        
        'Prewitt Y': np.array([
            [-1, -1, -1],
            [ 0,  0,  0],
            [ 1,  1,  1]
        ], dtype=np.float64),
        
        'Laplacien\n(4-connexe)': np.array([
            [ 0,  1,  0],
            [ 1, -4,  1],
            [ 0,  1,  0]
        ], dtype=np.float64),
        
        'Laplacien\n(8-connexe)': np.array([
            [ 1,  1,  1],
            [ 1, -8,  1],
            [ 1,  1,  1]
        ], dtype=np.float64),
        
        'Sharpen\n(netteté)': np.array([
            [ 0, -1,  0],
            [-1,  5, -1],
            [ 0, -1,  0]
        ], dtype=np.float64),
        
        'Sharpen\n(fort)': np.array([
            [-1, -1, -1],
            [-1,  9, -1],
            [-1, -1, -1]
        ], dtype=np.float64),
        
        'Emboss\n(relief)': np.array([
            [-2, -1, 0],
            [-1,  1, 1],
            [ 0,  1, 2]
        ], dtype=np.float64),
        
        'Emboss\n(haut)': np.array([
            [ 0,  1,  0],
            [ 0,  0,  0],
            [ 0, -1,  0]
        ], dtype=np.float64),
        
        'Roberts X\n(diagonal)': np.array([
            [1,  0],
            [0, -1]
        ], dtype=np.float64),
        
        'Roberts Y\n(diagonal)': np.array([
            [ 0, 1],
            [-1, 0]
        ], dtype=np.float64),
    }
    
    # =========================================================================
    # Fonction de convolution
    # =========================================================================
    
    def convoluer(image, noyau):
        """Applique une convolution avec padding par réflexion."""
        h, w = image.shape
        kh, kw = noyau.shape
        pad_h, pad_w = kh // 2, kw // 2
        
        img_pad = np.pad(image.astype(np.float64), 
                         ((pad_h, pad_h), (pad_w, pad_w)), 
                         mode='reflect')
        resultat = np.zeros((h, w), dtype=np.float64)
        
        for i in range(h):
            for j in range(w):
                resultat[i, j] = np.sum(img_pad[i:i+kh, j:j+kw] * noyau)
        
        return resultat
    
    # =========================================================================
    # Application de tous les noyaux
    # =========================================================================
    
    resultats = {}
    for nom, noyau in noyaux.items():
        resultats[nom] = convoluer(img, noyau)
    
    # =========================================================================
    # Figure 1 : Filtres de LISSAGE
    # =========================================================================
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    filtres_lissage = ['Identité', 'Moyenneur\n(box blur)', 'Gaussien\n3×3', 'Gaussien\n5×5']
    
    # Ligne 1 : Images
    for idx, nom in enumerate(filtres_lissage):
        ax = axes[0, idx]
        res = resultats[nom]
        ax.imshow(np.clip(res, 0, 255), cmap='gray', vmin=0, vmax=255)
        ax.set_title(nom, fontsize=11, fontweight='bold')
        ax.axis('off')
    
    # Ligne 2 : Noyaux
    for idx, nom in enumerate(filtres_lissage):
        ax = axes[1, idx]
        noyau = noyaux[nom]
        kh, kw = noyau.shape
        
        ax.imshow(noyau, cmap='Blues', vmin=0, vmax=noyau.max())
        for i in range(kh):
            for j in range(kw):
                val = noyau[i, j]
                if val != 0:
                    txt = f'{val:.2f}' if val < 1 else f'{val:.0f}'
                    ax.text(j, i, txt, ha='center', va='center', fontsize=8)
        ax.set_title(f'Σ = {noyau.sum():.2f}', fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle('FILTRES DE LISSAGE (réduction du bruit, flou)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # =========================================================================
    # Figure 2 : Filtres de CONTOURS (gradient)
    # =========================================================================
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    filtres_contours = ['Sobel X\n(contours V)', 'Sobel Y\n(contours H)', 
                        'Prewitt X', 'Prewitt Y']
    
    # Ligne 1 : Images
    for idx, nom in enumerate(filtres_contours):
        ax = axes[0, idx]
        res = resultats[nom]
        vmax = max(abs(res.min()), abs(res.max()))
        ax.imshow(res, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
        ax.set_title(nom, fontsize=11, fontweight='bold')
        ax.axis('off')
    
    # Ligne 2 : Noyaux
    for idx, nom in enumerate(filtres_contours):
        ax = axes[1, idx]
        noyau = noyaux[nom]
        kh, kw = noyau.shape
        
        ax.imshow(noyau, cmap='RdBu_r', vmin=-2, vmax=2)
        for i in range(kh):
            for j in range(kw):
                val = noyau[i, j]
                color = 'white' if abs(val) > 1 else 'black'
                ax.text(j, i, f'{int(val):+d}', ha='center', va='center', 
                       fontsize=12, fontweight='bold', color=color)
        ax.set_title(f'Σ = {noyau.sum():.0f}', fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle('FILTRES DE GRADIENT (détection de contours directionnels)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # =========================================================================
    # Figure 3 : Filtres LAPLACIEN (contours toutes directions)
    # =========================================================================
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    filtres_laplacien = ['Laplacien\n(4-connexe)', 'Laplacien\n(8-connexe)', 
                         'Roberts X\n(diagonal)', 'Roberts Y\n(diagonal)']
    
    # Ligne 1 : Images
    for idx, nom in enumerate(filtres_laplacien):
        ax = axes[0, idx]
        res = resultats[nom]
        vmax = max(abs(res.min()), abs(res.max()))
        ax.imshow(res, cmap='RdBu_r', vmin=-vmax, vmax=vmax)
        ax.set_title(nom, fontsize=11, fontweight='bold')
        ax.axis('off')
    
    # Ligne 2 : Noyaux
    for idx, nom in enumerate(filtres_laplacien):
        ax = axes[1, idx]
        noyau = noyaux[nom]
        kh, kw = noyau.shape
        
        vmax_k = max(abs(noyau.min()), abs(noyau.max()))
        ax.imshow(noyau, cmap='RdBu_r', vmin=-vmax_k, vmax=vmax_k)
        for i in range(kh):
            for j in range(kw):
                val = noyau[i, j]
                color = 'white' if abs(val) > vmax_k * 0.5 else 'black'
                ax.text(j, i, f'{int(val):+d}', ha='center', va='center', 
                       fontsize=12, fontweight='bold', color=color)
        ax.set_title(f'Σ = {noyau.sum():.0f}', fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle('FILTRES LAPLACIEN et ROBERTS (contours multi-directionnels)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # =========================================================================
    # Figure 4 : Filtres de NETTETÉ et RELIEF
    # =========================================================================
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    filtres_nettete = ['Sharpen\n(netteté)', 'Sharpen\n(fort)', 
                       'Emboss\n(relief)', 'Emboss\n(haut)']
    
    # Ligne 1 : Images
    for idx, nom in enumerate(filtres_nettete):
        ax = axes[0, idx]
        res = resultats[nom]
        # Pour emboss, centrer autour de 128
        if 'Emboss' in nom:
            res_affich = res + 128
        else:
            res_affich = res
        ax.imshow(np.clip(res_affich, 0, 255), cmap='gray', vmin=0, vmax=255)
        ax.set_title(nom, fontsize=11, fontweight='bold')
        ax.axis('off')
    
    # Ligne 2 : Noyaux
    for idx, nom in enumerate(filtres_nettete):
        ax = axes[1, idx]
        noyau = noyaux[nom]
        kh, kw = noyau.shape
        
        vmax_k = max(abs(noyau.min()), abs(noyau.max()))
        ax.imshow(noyau, cmap='RdBu_r', vmin=-vmax_k, vmax=vmax_k)
        for i in range(kh):
            for j in range(kw):
                val = noyau[i, j]
                color = 'white' if abs(val) > vmax_k * 0.5 else 'black'
                ax.text(j, i, f'{int(val):+d}', ha='center', va='center', 
                       fontsize=12, fontweight='bold', color=color)
        ax.set_title(f'Σ = {noyau.sum():.0f}', fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.suptitle('FILTRES DE NETTETÉ et RELIEF (accentuation des détails)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # =========================================================================
    # Figure 5 : Comparaison Sobel vs Magnitude
    # =========================================================================
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    Gx = resultats['Sobel X\n(contours V)']
    Gy = resultats['Sobel Y\n(contours H)']
    magnitude = np.sqrt(Gx**2 + Gy**2)
    
    axes[0].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[0].set_title('Image originale', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(np.abs(Gx), cmap='hot')
    axes[1].set_title('|Gx| (contours verticaux)', fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(np.abs(Gy), cmap='hot')
    axes[2].set_title('|Gy| (contours horizontaux)', fontweight='bold')
    axes[2].axis('off')
    
    axes[3].imshow(magnitude, cmap='hot')
    axes[3].set_title('Magnitude √(Gx² + Gy²)\n(tous les contours)', fontweight='bold')
    axes[3].axis('off')
    
    plt.suptitle('COMBINAISON DES GRADIENTS SOBEL', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
  
demo_tous_les_noyaux()

##### Récapitulatif filtres de convolution

| CATÉGORIE          | SOMME    | EFFET                                  |
|--------------------|----------|----------------------------------------|
|                    |  LISSAGE |                                        |
|   Moyenneur        |    1     | Flou simple, réduit le bruit           |
|   Gaussien         |    1     | Flou naturel, préserve les structures  |
|                    | GRADIENT |                                        |
|   Sobel            |    0     | Contours directionnels, robuste        |
|   Prewitt          |    0     | Contours directionnels, simple         |
|   Roberts          |    0     | Contours diagonaux, rapide (2×2)       |
|                    |LAPLACIEN |                                        |
|   4-connexe        |    0     | Contours toutes directions (dérivée 2) |
|   8-connexe        |    0     | Plus sensible, détecte plus de détails |
|                    |  NETTETÉ |                                        |
|   Sharpen          |    1     | Accentue les contours                  |
|   Sharpen fort     |    1     | Accentuation agressive                 |
|                    |   RELIEF |                                        |
|   Emboss           |   ~1     | Effet 3D, éclairage directionnel       |
    
RÈGLE IMPORTANTE :
* Σ coefficients = 1  →  préserve la luminosité (flou, netteté)
* Σ coefficients = 0  →  extrait les variations (contours, gradient)

### 4.2 : Limites de la simple convolution

Malgré leurs avantages, au nombre duquel la grande variété de traitements qu’ils peuvent réaliser, les noyaux de convolution ne sont pas sans limites.

Nous avons vu que cette méthode correspondait à un *filtrage local* de l’image : le traitement d’un pixel dépend des valeurs de ses voisins. Par exemple si l’on souhaite créer un filtre qui réduit le bruit d’un document (cas typique : texte scanné, il y a toujours des poussières sur la vitre du scan), ce qui va résulter en des petits points qui vont apparaître blancs ou noirs en fonction de l’éclairage, de la couleur qu’ils retrouvent, etc., pour « gommer » ces points, on va appliquer un flou qui préservera les structures dans l’ensemble (important pour des lettres), par exemple un flou gaussien. C’est comme si on appliquait une éponge mouillée qui va fondre ou égaliser localement des zones (ensemble de pixels) de valeurs différentes par un effet d’estompage.

Malgré les qualités du filtre gaussien, cette approche n’en demeure pas moins destructive. Dans le cas de lettres, où chaque détail compte pour les différencier (cf. `n` vs. `m`, `u` vs. `v` vs `w`, ou encore `r` vs. `t`), cela peut être gênant, ou tout au moins toutes les applications où la préservation des détails est intéressantes.

Voici ci-dessous une illustration du phénomène. On simule une situation de texte scanné avec du bruit en créant des rectangles plus ou moins fins, une teinte grise du fond (ce qui arrive souvent dans les scans) et l’ajout d’un bruit aléatoire (pixels blancs et noirs). Pour mesurer l’effet des transformations on décompte le nombre de composantes (formes indépendantes : rectangles, pixels/bruit…) : cela permetra de voir l’efficacité du nettoyage.

Nous allons comparer 3 méthodes pour éliminer le bruit :

- un seuillage simple qui élimine **indifféremment tous** les pixels en dessous d’une certaine valeur
- un seuillage + flou gaussier (convolution) qui calcule une **moyenne pondérée** des pixels voisins
- un seuillage + filtre morphologique. Nous n’avons pas encore vu cette dernière méthode, c’est l’objet de la section suivante, mais nous la présentons ici en guise d’introduction, ce qui permet déjà de vous donner une mesure de son efficacité. Sachez simplement que cette méthode consiste simplement à comparer chaque composante de l’image avec un motif (forme) spécifique que l’on appelle un **élément structurant**. À l’issu de la comparaison on décide s’il faut éliminer ou conserver la composante en question. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def creer_document_test():
    """
    Crée une image simulant un document scanné :
    - Fond blanc (240)
    - "Lettres" = rectangles noirs de tailles variées
    - Bruit = points FINS isolés (1 pixel)
    """
    np.random.seed(42)
    h, w = 150, 300
    
    # Fond légèrement gris (comme un scan)
    img = np.ones((h, w), dtype=np.uint8) * 240
    
    # Ligne 1 : grandes lettres
    positions_l1 = [(20, 20), (20, 50), (20, 80), (20, 110), (20, 140)]
    for (y, x) in positions_l1:
        letter_h, letter_w = np.random.randint(28, 38), np.random.randint(12, 20)
        img[y:y+letter_h, x:x+letter_w] = np.random.randint(20, 50)
    
    # Ligne 2 : lettres moyennes
    positions_l2 = [(70, 20), (70, 45), (70, 70), (70, 95), (70, 120), (70, 145)]
    for (y, x) in positions_l2:
        letter_h, letter_w = np.random.randint(20, 28), np.random.randint(10, 16)
        img[y:y+letter_h, x:x+letter_w] = np.random.randint(20, 50)
    
    # Ligne 3 : petites lettres (mais pas trop petites)
    positions_l3 = [(110, 20 + i*18) for i in range(8)]
    for (y, x) in positions_l3:
        letter_h, letter_w = np.random.randint(15, 22), np.random.randint(8, 14)
        img[y:y+letter_h, x:x+letter_w] = np.random.randint(20, 50)
    
    # Zone avec un rectangle plein (comme un logo ou un cadre)
    img[20:90, 200:290] = 30
    img[30:80, 210:280] = 240  # Trou au milieu
    
    # BRUIT FIN : points isolés de 1 pixel uniquement
    bruit_coords = []
    for _ in range(300):
        y, x = np.random.randint(0, h), np.random.randint(0, w)
        img[y, x] = np.random.randint(0, 60)  # Point noir isolé
        bruit_coords.append((y, x))
    
    # Quelques points blancs (sel) - aussi 1 pixel
    for _ in range(100):
        y, x = np.random.randint(0, h), np.random.randint(0, w)
        img[y, x] = 255
    
    return img


def creer_element_structurant_carre(taille):
    return np.ones((taille, taille), dtype=np.uint8)


def erosion(img_bin, es):
    """Érosion : pixel blanc si TOUS les voisins sont blancs."""
    h, w = img_bin.shape
    es_h, es_w = es.shape
    pad = es_h // 2
    
    img_bool = (img_bin > 0)
    img_pad = np.pad(img_bool, pad, mode='constant', constant_values=False)
    resultat = np.zeros((h, w), dtype=bool)
    
    for i in range(h):
        for j in range(w):
            region = img_pad[i:i+es_h, j:j+es_w]
            resultat[i, j] = np.all(region[es == 1])
    
    return (resultat * 255).astype(np.uint8)


def dilatation(img_bin, es):
    """Dilatation : pixel blanc si AU MOINS UN voisin est blanc."""
    h, w = img_bin.shape
    es_h, es_w = es.shape
    pad = es_h // 2
    
    img_bool = (img_bin > 0)
    img_pad = np.pad(img_bool, pad, mode='constant', constant_values=False)
    resultat = np.zeros((h, w), dtype=bool)
    
    for i in range(h):
        for j in range(w):
            region = img_pad[i:i+es_h, j:j+es_w]
            resultat[i, j] = np.any(region[es == 1])
    
    return (resultat * 255).astype(np.uint8)


def ouverture(img_bin, es):
    """Ouverture = érosion puis dilatation."""
    return dilatation(erosion(img_bin, es), es)


def fermeture(img_bin, es):
    """Fermeture = dilatation puis érosion."""
    return erosion(dilatation(img_bin, es), es)


def compter_composantes_connexes(img_bin):
    """
    Compte le nombre de composantes connexes (objets) dans une image binaire.
    Utilise un algorithme de flood-fill simplifié.
    """
    img = (img_bin > 0).astype(np.int32)
    h, w = img.shape
    visited = np.zeros_like(img, dtype=bool)
    n_composantes = 0
    
    def flood_fill(start_i, start_j):
        stack = [(start_i, start_j)]
        while stack:
            i, j = stack.pop()
            if i < 0 or i >= h or j < 0 or j >= w:
                continue
            if visited[i, j] or img[i, j] == 0:
                continue
            visited[i, j] = True
            stack.extend([(i+1, j), (i-1, j), (i, j+1), (i, j-1)])
    
    for i in range(h):
        for j in range(w):
            if img[i, j] > 0 and not visited[i, j]:
                flood_fill(i, j)
                n_composantes += 1
    
    return n_composantes


def mesurer_qualite(img_bin, nom=""):
    """
    Mesure la qualité d'une image binarisée.
    
    Métriques :
    - Nombre de composantes connexes (beaucoup = bruit résiduel)
    - Ratio pixels noirs / total
    """
    n_composantes = compter_composantes_connexes(255 - img_bin)  # Compter les objets noirs
    n_pixels_noirs = np.sum(img_bin == 0)
    ratio_noir = n_pixels_noirs / img_bin.size * 100
    
    return {
        'nom': nom,
        'composantes': n_composantes,
        'pixels_noirs': n_pixels_noirs,
        'ratio_noir': ratio_noir
    }


# =============================================================================
# MÉTHODES DE NETTOYAGE
# =============================================================================

def nettoyer_par_convolution(img, taille_noyau=5):
    """Nettoyage par flou gaussien + seuillage."""
    
    def noyau_gaussien(taille, sigma=1.0):
        ax = np.arange(taille) - taille // 2
        xx, yy = np.meshgrid(ax, ax)
        kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
        return kernel / kernel.sum()
    
    def convoluer(img, noyau):
        h, w = img.shape
        kh, kw = noyau.shape
        pad = kh // 2
        img_pad = np.pad(img.astype(float), pad, mode='reflect')
        resultat = np.zeros_like(img, dtype=float)
        for i in range(h):
            for j in range(w):
                resultat[i, j] = np.sum(img_pad[i:i+kh, j:j+kw] * noyau)
        return resultat
    
    noyau = noyau_gaussien(taille_noyau, sigma=taille_noyau/3)
    img_floue = convoluer(img, noyau)
    
    seuil = 128
    img_bin = np.where(img_floue < seuil, 0, 255).astype(np.uint8)
    
    return img_floue.astype(np.uint8), img_bin


def nettoyer_par_morphologie(img, taille_es=3):
    """Nettoyage par seuillage + ouverture + fermeture."""
    
    seuil = 128
    img_bin = np.where(img < seuil, 0, 255).astype(np.uint8)
    
    # Travailler sur l'image inversée (texte blanc sur fond noir)
    img_inv = 255 - img_bin
    
    es = creer_element_structurant_carre(taille_es)
    
    # Ouverture : supprime les petits points blancs (bruit)
    img_ouv = ouverture(img_inv, es)
    
    # Fermeture : reconnecte les petites cassures
    img_ferm = fermeture(img_ouv, es)
    
    # Ré-inverser
    img_finale = 255 - img_ferm
    
    return img_bin, img_finale


# =============================================================================
# EXÉCUTION ET COMPARAISON
# =============================================================================

img_originale = creer_document_test()

# Méthode 1 : Convolution
img_floue, img_conv = nettoyer_par_convolution(img_originale, taille_noyau=5)

# Méthode 2 : Morphologie
img_seuil, img_morpho = nettoyer_par_morphologie(img_originale, taille_es=3)

# =============================================================================
# MESURES OBJECTIVES
# =============================================================================

stats_seuil = mesurer_qualite(img_seuil, "Seuillage seul")
stats_conv = mesurer_qualite(img_conv, "Convolution")
stats_morpho = mesurer_qualite(img_morpho, "Morphologie")

print("=" * 70)
print("MESURE OBJECTIVE : NOMBRE DE COMPOSANTES CONNEXES")
print("=" * 70)
print("""
Un document propre devrait avoir peu de composantes connexes (une par lettre/forme).
Le bruit ajoute de nombreuses petites composantes parasites.
""")
print(f"{'Méthode':<20} {'Composantes':<15} {'Interprétation'}")
print("-" * 70)
print(f"{'Seuillage seul':<20} {stats_seuil['composantes']:<15} ← Beaucoup de bruit résiduel")
print(f"{'Convolution + seuil':<20} {stats_conv['composantes']:<15} ← Bruit atténué mais texte dégradé")
print(f"{'Morphologie':<20} {stats_morpho['composantes']:<15} ← Bruit supprimé, texte préservé ✓")
print()

# =============================================================================
# VISUALISATION
# =============================================================================

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Ligne 1 : Vue globale
axes[0, 0].imshow(img_originale, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original\n(avec bruit fin)', fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_seuil, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title(f'Seuillage seul\n({stats_seuil["composantes"]} composantes)', fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_conv, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title(f'Convolution\n({stats_conv["composantes"]} composantes)', fontweight='bold')
axes[0, 2].axis('off')

axes[0, 3].imshow(img_morpho, cmap='gray', vmin=0, vmax=255)
axes[0, 3].set_title(f'Morphologie\n({stats_morpho["composantes"]} composantes)', fontweight='bold', color='green')
axes[0, 3].axis('off')

# Ligne 2 : Zoom sur les petites lettres
y1, y2, x1, x2 = 100, 140, 15, 120

axes[1, 0].imshow(img_originale[y1:y2, x1:x2], cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title('Zoom original', fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_seuil[y1:y2, x1:x2], cmap='gray', vmin=0, vmax=255)
axes[1, 1].set_title('Zoom seuillage\n⚠️ Bruit visible', fontweight='bold', color='red')
axes[1, 1].axis('off')

axes[1, 2].imshow(img_conv[y1:y2, x1:x2], cmap='gray', vmin=0, vmax=255)
axes[1, 2].set_title('Zoom convolution\n⚠️ Lettres floues', fontweight='bold', color='orange')
axes[1, 2].axis('off')

axes[1, 3].imshow(img_morpho[y1:y2, x1:x2], cmap='gray', vmin=0, vmax=255)
axes[1, 3].set_title('Zoom morphologie\n✓ Propre et net', fontweight='bold', color='green')
axes[1, 3].axis('off')

plt.suptitle('CONVOLUTION vs MORPHOLOGIE : Nettoyage de document', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


On constate :

- le seuillage simple : il n’a aucun effet sur le bruit qui entre dans l’intervalle de valeurs tolérées

- Le flou gaussien :
    - ici le bruit est supprimé
    - ne pas perdre de vue que parfois le bruit – notamment s’il possède des composantes de grandes tailles – est simpleemnt atténué
    - limite : les contours fins sont aussi atténués ! (les formes sont rognées)

- l’approche  morphologique :
    - ici le bruit est supprimé
    - les composantes d’intérêt (qui simulent des lettres) sont parfaitement conservées (intactes)
    - attention si le bruit contient des composantes de taille suffisantes, elles vont être conservées telles quelles car identifiées comme des composantes à conserver

Si un filtre flou par convolution revient à tout estomper comme si on passer une éponge pour « nettoyer » le bruit, l’approce morphologique correspond plus à passer les composantes dans une sorte de tamis ou crible : les composantes seront triées selon leur taille et les plus petites éliminées.

Passons à une présentation plus détaillée de la morphologie mathématique, discipline sur laquelle se fonde la méthode des filtres morphologiques.

## 5. Morphologie Mathématique

### 5.1 Introduction

On peut distinguer deux manières de traiter les images : soit par une approche de **traitement du signal**, soit par une approche **ensembliste**. C’est à dire qu’on va construire des ensembles particuliers pour analyser ou traiter une image, et on va regarder si un pixel appartient ou pas à un ensemble donné.

L’approche morphologique appartient à la seconde famille, l’approche ensembliste. Il s’agit d’un traitement non-linéaire de l’information développée par **Georges Matheron** et **Jean Serra** (années 50/60 pour les bases de la théorie).
* elle offre un cadre mathématiquement accessible et très élégant au traitement de l’image sans apprentissage
* 1956 : application aux images binaires (améliorer la segmentation)
* 1978 : application aux images en niveaux de gris (segmentation + filtrage/suppression du bruit, détection de contour, amélioration du contraste, etc.). Pour traiter une image en couleur, il suffit d’appliquer séparément ces opérations sur chaque canal.

Aujourd’hui, quand on parle traitement d’image, le *deep learning* règne en maître. Ça n’interdit pas de s’intéresser à une méthode élégante qui a brillé pendant 40 ans, efficace sans demander d’énormes ressources (sobriété), et qui enrichit dans tous les cas notre boîte à outil. D’autant que les filtres morphologiques sont toujours pertinents pour prétraiter des images qui peuvent ensuite être traitées plus efficacement par des réseaux de neurone (ou vice-versa). On développe également des réseaux de neurones avec des couches morphologiques.

Des bibliothèques de traitgement d’image comme OpenCV ou Scikit-Image contiennent des méthodes implémentant des opérations de morphologie. Il s’agit donc d’une méthode à connaître lorsque l’on souhaite utiliser ces bibliothèques.

#### 5.1.1 Un peu d’algèbre, encore (mais pas beaucoup !)

Quel rapport avec l’algèbre ????
Hé bien… revenons à la convolution…

* L’opérateur réalisant la convolution par un noyau (kernel) est en fait un opérateur linéaire (par rapport à la translation, cf. [Blusseau & Puybareau, 2023](https://hal.science/hal-04148876/file/ejcim_2023_chap3.pdf))

Revenons aux concepts (il s’agit juste de quelques pistes intuitives, cf. la référence précédente pour une approche rigoureuse) : 

* En algèbre linéaire on a construit toute la théorie sur les espaces vectoriels en s’appuyant sur les opérations « somme » et « produit ». En morphologie on peut tout à fait remplacer la somme par les opération supremum ($>$) ou infimum ($<$) et l’opération $·$ par $+$. Les structures considérées ne seront plus des espaces vectoriels, mais des « treillis complets ».

* En algèbre linéaire, nous avons vu que les combinaisons linéaires (= somme de produits) étaient des opérations qui jouent un rôle central. Par ailleurs, dans le cas du filtrage, l’application d’un noyau de convolution à une image consiste à calculer une combinaison linéaire pour chaque pixel. On va créer des opérations qui vont jouer un rôle tout aussi central dans le cadre de la morphologie mathématique, cette fois avec supremum, infinum et $+$. Ces opération seront respectivement la dilation (δ) et l’érosion (ε). Elle seront calculées pour chaque pixel d’une image en déplaçant de la même manière que pour la convolution un masque sur l’image, en regardant cette fois pour chaque pixel si le masque positionné sur ce pixel recouvre pour partie d’autre pixels d’une forme (dilatation) ou du fond (érosion). Si tel est le cas on modifiera la valeur du pixel sur lequel le masque est positionné en la valeur maximale (dilatation) ou minimale (érosion) des pixels appartenant au masque.

Pas de panique, comme d’habitude c’est une opération plus difficile à décrire qu’à réaliser, un schéma explicatif ci-dessous devrait vous éclairer. Pour le moment, retenez surtout ce résumé :

* Algèbre linéaire -> espace vectoriels, opérations (+, ·) -> combinaisons linéaires 

* Morphologie -> treillis complets, opérations (infinum, supremum, +) -> dilation ou érosion

Dans le cas des filtres morphologiques, la forme du masque va avoir une importance capitale, on va appeler ces formes des **éléments structurants** (ES). 

Pour l’illustration de ces concepts, on va par soucis de simplicité se positionner dans le cas d’images binaires (un pixel est blanc ou noir), nous indiquerons via des notes les adaptations qu’implique le traitement d’images en niveau de gris.

Comme d’habitude encore, on donnera d’abord une définition intuitive, puis une définition plus formelle.

##### Éléments structurants, érosion et dilation

###### Définition intuitive/pratique

On peut se représenter un élément structurant (ES) comme un masque ayant une petite forme géométrique (carré, disque, croix...) que l'on déplace sur l'image pour « sonder » les structures (formes) présentes sur l’image. L’élément structurant est définit par trois caractéristique :
- sa taille
- sa forme
- son origine (que l’on place en général au centre)

La taille de l’ES détermine l'échelle des détails affectés : dans le cas de l’érosion par exemple tout objet plus petit que l'élément structurant pourra être supprimé, tandis que les objets plus grands (qui peuvent contenir l’ES) seront préservés.

Les ES les plus courants sont :

![Exemples d’élements structurants](images/ElementsStructurants.png)

Comment on procède : on déplace l’ES sur l’image, et on va modifier ou non le pixel situé à l’origine de l’ES (on place la plupart du temps l’origine au centre mais on pourrait en décider autrement pour certaines applications) en fonction de ce que l’ES recouvre sur l’image :

![Exemple recouvrement ES - Formes](images/ES-Formes.png)

Opérations :
  
- **Dilatation :**

![schéma érosion](images/Dilatation.png)

- **Erosion :**

![schéma érosion](images/Erosion.png)

Il s’agit des deux opération de base qui nous  permettrons de définir d’autres opérations (ouverture, fermeture, gradient interne, externe, morphologique, etc.) que nous implémenterons dans la suite. 

###### Définition formelle

Un élément structurant $B$ est un ensemble de points (un sous-ensemble de $\mathbb{Z}^2$ pour une image discrète) défini par rapport à une origine, généralement son centre. Formellement :

$$
B = \{b_1, b_2, \ldots, b_n\} \subset \mathbb{Z}^2
$$
où chaque $b_i = (x_i, y_i)$ représente un décalage relatif par rapport à l'origine.

Lorsqu'on positionne l'élément structurant en un point $p = (x, y)$ de l'image, on obtient le translaté de $B$ en $p$ :

$$
B_p = \{p + b \mid b \in B\} = \{(x + x_i, y + y_i) \mid (x_i, y_i) \in B\}
$$

Les opérations morphologiques sont alors définies comme des relations ensemblistes entre ce translaté $B_p$ et l'ensemble $X$ des pixels de l'objet dans l'image :

* Érosion : $\varepsilon_B(X) = \{p \mid B_p \subseteq X\}$ (tous les points où $B$ est entièrement inclus dans $X$)
* Dilatation : $\delta_B(X) = \{p \mid B_p \cap X \neq \emptyset\}$ (tous les points où $B$ touche $X$)

Nous allons manipuler des formes constituées de points (pixels) positionnées sur un réseau discret, et déterminer si une forme est à l’intérieur ou non d’une autre forme donnée. C’est l’objet de la [géométrie des nombres](https://fr.wikipedia.org/wiki/G%C3%A9om%C3%A9trie_des_nombres), fondée par [Hermann Minkowski](https://fr.wikipedia.org/wiki/Hermann_Minkowski) - accessoirement aussi celui qui a théorisé le continuum espace/temps. Nous allons d’ailleurs utiliser les opérations de Minkowski pour représenter les opérations auxquelles nous allons procéder :

* **Soustraction de Minkowski ⊖**
L'opération $A \ominus B$ est l'ensemble des points $p$ tels que le translaté de (la forme) $B$ en $p$ soit entièrement contenu dans (la forme) $A$ :

$$
A \ominus B = \{p \mid B_p \subseteq A\}
$$
L'érosion de $A$ par $B$ correspond exactement à cette opération : $\varepsilon_B(A) = A \ominus B$, ce qui revient à ne conserver que les pixels où l'élément structurant "rentre" entièrement dans l'objet.

* **Addition de Minkowski**
L'opération $A \oplus B$ est l'ensemble des points $p$ tels que le translaté de $B$ en $p$ ait une intersection non vide avec $A$ :

$$
A \oplus B = \{p \mid B_p \cap A \neq \emptyset\}
$$
La dilatation de $A$ par $B$ correspond exactement à cette opération : $\delta_B(A) = A \oplus B$, ce qui revient à marquer comme "objet" tout pixel où l'élément structurant "touche" au moins un pixel de l'objet original.

### 5.2 Maintenant un peu de code : éléments structurants

Le bout de code ci-dessous montre comment créer certains ES de manière procédurale :

In [ ]:
def creer_element_structurant(forme, taille):
    """Crée un élément structurant."""
    if taille % 2 == 0: taille += 1
    centre = taille // 2
    
    if forme == 'carre':
        return np.ones((taille, taille), dtype=np.uint8)
    elif forme == 'croix':
        es = np.zeros((taille, taille), dtype=np.uint8)
        es[centre, :] = 1
        es[:, centre] = 1
        return es
    elif forme == 'disque':
        y, x = np.ogrid[:taille, :taille]
        return ((x - centre)**2 + (y - centre)**2 <= centre**2).astype(np.uint8)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, forme in zip(axes, ['carre', 'croix', 'disque']):
    es = creer_element_structurant(forme, 7)
    ax.imshow(es, cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'{forme.capitalize()} 7x7')
    ax.axis('off')
plt.tight_layout()
plt.show()

### 5.3 Érosion

$A \ominus B$ : Un pixel vaut 1 *seulement si* **TOUS** les pixels sous l'ES valent 1 = un pixel vaut 0 si **au moins** un pixel vaut 0.

**Effets** : Amincit les objets, supprime le bruit.

In [ ]:
def erosion(img_binaire, element_structurant):
    """Érosion morphologique."""
    h, w = img_binaire.shape
    es_h, es_w = element_structurant.shape
    pad_h, pad_w = es_h // 2, es_w // 2
    
    img_bin = (img_binaire > 0).astype(np.uint8)
    img_padded = np.pad(img_bin, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    resultat = np.zeros((h, w), dtype=np.uint8)
    
    for i in range(h):
        for j in range(w):
            region = img_padded[i:i+es_h, j:j+es_w]
            if np.all(region[element_structurant == 1] == 1):
                resultat[i, j] = 1
    return (resultat * 255).astype(np.uint8)

Nous allons appliquer cette transformation à des formes ci-dessous, mais pour comparer avec la dilation implémentons d’abord cette dernière :

### 5.4 Dilatation

$A \oplus B$ : Un pixel vaut 1 si **AU MOINS UN** pixel sous l'ES vaut 1.

**Effets** : Épaissit les objets, remplit les trous.

In [ ]:
def dilatation(img_binaire, element_structurant):
    """Dilatation morphologique."""
    h, w = img_binaire.shape
    es_h, es_w = element_structurant.shape
    pad_h, pad_w = es_h // 2, es_w // 2
    
    img_bin = (img_binaire > 0).astype(np.uint8)
    img_padded = np.pad(img_bin, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    resultat = np.zeros((h, w), dtype=np.uint8)
    
    for i in range(h):
        for j in range(w):
            region = img_padded[i:i+es_h, j:j+es_w]
            if np.any(region[element_structurant == 1] == 1):
                resultat[i, j] = 1
    return (resultat * 255).astype(np.uint8)

Maintenant que nous avons deux fonctions qui implémentent ces deux opérations de base de la morphologie, testons les sur une image test contenant des formes (dont du bruit) :

In [ ]:
# Image test
def creer_image_test_morpho():
    img = np.zeros((150, 200), dtype=np.uint8)
    # Grand rectangle
    img[20:60, 20:80] = 255
    # Cercle
    y, x = np.ogrid[:150, :200]
    img[(x - 140)**2 + (y - 40)**2 <= 625] = 255
    # Deux petits carrés avec trous au centre
    img[80:100, 30:50] = 255
    img[87:93, 37:43] = 0  # Trou dans le premier carré
    img[80:100, 55:75] = 255
    img[88:92, 63:67] = 0  # Trou dans le second carré
    # Grand carré avec trou
    img[80:130, 110:160] = 255
    img[95:115, 125:145] = 0
    # Bruit
    for _ in range(30):
        by, bx = np.random.randint(0, 150), np.random.randint(0, 200)
        if img[by, bx] == 0:
            img[max(0,by-1):by+2, max(0,bx-1):bx+2] = 255
    return img

img_test = creer_image_test_morpho()
es3 = creer_element_structurant('carre', 5)

img_erodee = erosion(img_test, es3)
img_dilatee = dilatation(img_test, es3)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
axes[0].imshow(img_test, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(img_erodee, cmap='gray'); axes[1].set_title('Érosion'); axes[1].axis('off')
axes[2].imshow(img_dilatee, cmap='gray'); axes[2].set_title('Dilatation'); axes[2].axis('off')
plt.tight_layout()
plt.show()

Ces transformations sont intéressantes, mais si elles suppriment le bruit (dans le cadre de l’érosion) ou l’amplifie (dilatation), elles modifient également les formes qui nous intéressent, ce qui était une limitation des filtres de convolution. Nous allons voir qu’en combinant adroitement ces deux opérations pour créer de nouvelles opérations (ouverture, fermeture, etc.), on peut réaliser des transformations plus précises et ciblées.

### 5.5 Combinaisons d’opérations morphologique

Nous allons dans cette section passer en revue différentes manières de combiner les opérations morphologiques élémentaires que sont l’érosion et la dilation, et voir quelles peuvent être leurs applications (effets).

#### 5.5.1 Ouverture et fermeture

##### Ouverture

Si l’érosion supprime le bruit en éliminant toutes les composantes (formes) plus petites que l’ES, mais qu’elle érode de la même manière les formes plus grandes, on peut résoudre le problème en faisant plusieurs passes avec des opérations qui s’annulent :

- 1. On applique une érosion pour supprimer le bruit
  2. On applique ensuite une dilatation pour contrebalancer l’érosion des formes qui seront toujours présentent sur l’image (trop grandes pour avoir été éliminées). Vu que les plus petits éléments auront déjà été supprimés par l’érosion, la dilation ne risque pas d’amplifier le bruit.

Cela marche surtout pour le bruit sur le fond.

On appelle cette combinaison une ouverture :
**Ouverture** : $(A \ominus B) \oplus B$ → Supprime le bruit

##### Fermeture

On peut se demander ce qu’il se passe si on fait l’opération inverse : une dilatation suivie d’une érosion. Pour ce qui est du bruit, celui-ci sera amplifié par la dilatation, et il reviendra à sa valeur de départ si on applique ensuite une érosion : l’effet sera nul. Par contre, si on prend garde au fait que l’érosion ne va s’appliquer que là où le fond apparaît à proximité, si une dilation « bouche un trou » (comme sur un des petits carrés dans notre exemple), dans ce cas une érosion subséquente ne pourra pas le faire réapparaître. Cette opération va donc être appelée une fermeture, car elle va refermer les petits vides ou creux dans les formes. C’est un moyen par exemple de gérer la composante du bruit qui recouvre les formes (et non le fond).

On constate par la même occasion que l’ordre dans lequel on applique érosion et dilatation a son importance : pas de commutativité.

**Fermeture** : $(A \oplus B) \ominus B$ → Remplit les trous

##### Exercice  
Voyons ce que cela donne sur notre exemple : créez une fonction ouverture et une fonction fermeture à partir des fonctions `dilation()` et `erosion()` déjà créées.

Testez ensuite plussieurs ES différents (fomres, tailles…)

In [ ]:
def ouverture(img, es):
    return dilatation(erosion(img, es), es)

def fermeture(img, es):
    return erosion(dilatation(img, es), es)

es_test = creer_element_structurant('carre', 5)

img_ouv = ouverture(img_test, es_test)
img_ferm = fermeture(img_test, es_test)
img_nettoye = fermeture(ouverture(img_test, es3), es3)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes[0,0].imshow(img_test, cmap='gray'); axes[0,0].set_title('Original'); axes[0,0].axis('off')
axes[0,1].imshow(img_ouv, cmap='gray'); axes[0,1].set_title('Ouverture'); axes[0,1].axis('off')
axes[1,0].imshow(img_ferm, cmap='gray'); axes[1,0].set_title('Fermeture'); axes[1,0].axis('off')
axes[1,1].imshow(img_nettoye, cmap='gray'); axes[1,1].set_title('Ouverture + Fermeture'); axes[1,1].axis('off')
plt.tight_layout()
plt.show()

#### 5.5.2 Gradient morphologique

Pour résumer grossièreemnt, une dilatation dilates les contours, et une érosion les érode. On peut donc isoler ces contours en combinant ces deux opérations en faisant leur différence : Si on procède à une dilatation, on obtient une image avec les contours contours épaissis. Si l’on procède à une érosion, on obtient une image avec les contours réduits. Si on soustrait cette seconde image à la première, ce qui va rester c’est la différence entre les deux images, donc la zone où les contours ont été dilatés/érodés : on aura extrait les contours.

**Gradient** = Image dilatée - Image érodée → extraction de contour

##### Exercice

Réalisez une extraction de contour à l’aide d’un gradient morphologique.
Pour normaliser l’image résultante, vous pouvez utiliserla méthode `np.clip()` (cherchez dans la doc).

In [ ]:
def gradient_morphologique(img, es):
    return np.clip(dilatation(img, es).astype(int) - erosion(img, es).astype(int), 0, 255).astype(np.uint8)

grad = gradient_morphologique(img_test, es3)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img_test, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(grad, cmap='gray'); axes[1].set_title('Gradient morphologique'); axes[1].axis('off')
plt.tight_layout()
plt.show()

#### 5.5.3 Top-Hat et Black-Hat

##### Top-hat
Creusant la piste selon laquelle on peut créer des opéarations morphologiques en soustrayans des images entre elles, si réaliser une ouverture supprime le bruit (en fait les petits détails clairs sur le fond), si on fait la différence entre une image non-transformée et la même image après ouverture, on devrait sélectionner spécifiquement ce bruit (vu que les formes de bonne taille seront les éléments communs entre les deux images, qui seront alors supprimé par la soustraction).

On désigne par *top-hat* (« chapeau haut-de-forme ») la différence entre l'image originale et son ouverture : 

$\text{TopHat}(A) = A - (A \circ B)$ où $\circ$ désigne l'ouverture morphologique.

Cette opération permet d'extraire les petits éléments clairs (plus petits que l'élément structurant) qui disparaissent lors de l'ouverture, ce qui est particulièrement utile pour détecter des détails fins sur un fond non uniforme ou pour corriger des variations d'éclairage avant une binarisation.

##### Exercice

Implémentez un *top-hat* et tester l’effet de différents éléments structurant.

In [ ]:
def top_hat(img, es):
    return np.clip(img.astype(int) - ouverture(img, es).astype(int), 0, 255).astype(np.uint8)

es7 = creer_element_structurant('carre', 7)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].imshow(img_test, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(top_hat(img_test, es7), cmap='gray'); axes[1].set_title('Top-Hat (bruit)'); axes[1].axis('off')
plt.tight_layout()
plt.show()

##### Black-hat
Immédiatement on a envie de tester ce que donne la différence entre la fermeture d’une image et l’image originale :

$\text{BlackHat}(A) = (A \bullet B) - A$ où $\bullet$ désigne la fermeture morphologique.

La fermeture va combler les petits éléments de couleur du fond dont la taille est inférieure à celle de l’ES. Les formes resteront intactes. En faisant la différence, là où la fermeture a comblé la forme, la couleur sera celel de la forme et non du fond, donc ladifférence fera apparaîre ces zones comblées (et donc les défauts).

Cette opération permet d'extraire les petits éléments sombres (trous, fissures, cavités plus petits que l'élément structurant) qui sont comblés lors de la fermeture, ce qui est utile pour détecter des défauts, des rayures ou du texte sombre sur un fond clair dans le cas de l’OCR par exemple, mais ça peut être utile dans d’autres domaines : détection de fissures dans du métal (inspection de soudures, corrosion…), défaut d’impression (contrôle qualité), détecter des craquelures dans une peinture (restaurantion, conservation dans des musées, etc.)

##### Exercice

1. Implémentez un *black-hat* et tester l’effet de différents éléments structurant.

In [ ]:
def black_hat(img, es):
    return np.clip(fermeture(img, es).astype(int) - img.astype(int), 0, 255).astype(np.uint8)

es5 = creer_element_structurant('carre', 4)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].imshow(img_test, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(black_hat(img_test, es5), cmap='gray'); axes[1].set_title('Black-Hat (trous)'); axes[1].axis('off')
plt.tight_layout()
plt.show()

L’image utilisé pour le test est peu adaptée pour mettre en valeur l’opération black-hat : il n‘y a pas vraiment de défauts dans les formes.

2. Voici une fonction qui génère des formes qui ressemblent à des lettres, avec des défauts dedans. Appliquer un black-hat pour sélectionner ces défauts :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def creer_image_test_blackhat():
    """
    Crée une image simulant un document ou une surface avec :
    - Fond clair
    - Grandes formes sombres (lettres/logos)
    - Défauts = fissures/rayures CLAIRES (couleur du fond) dans les formes sombres
    """
    np.random.seed(42)
    h, w = 180, 280
    
    # Fond noir
    img = np.zeros((h, w), dtype=np.float64)
    
    # =========================================================================
    # GRANDES FORMES SOMBRES (lettres stylisées)
    # =========================================================================
    
    # Lettre "H"
    img[30:130, 30:50] = 255      # Jambe gauche
    img[30:130, 90:110] = 255     # Jambe droite
    img[70:85, 30:110] = 255      # Barre horizontale
    
    # Lettre "I"
    img[30:45, 130:190] = 255     # Barre du haut
    img[115:130, 130:190] = 255   # Barre du bas
    img[30:130, 152:168] = 255    # Jambe verticale
    
    # Carré plein
    img[30:130, 210:260] = 255
    
    # =========================================================================
    # DÉFAUTS CLAIRS (fissures, rayures = couleur du fond qui traverse les formes)
    # =========================================================================
    
    # Rayure horizontale à travers le H
    img[60:63, 30:110] = 0
    
    # Rayure verticale à travers la barre du H
    img[70:85, 65:68] = 0
    
    # Fissure horizontale à travers le I
    img[80:82, 152:168] = 0
    
    # Rayure à travers la barre du haut du I
    img[36:39, 130:190] = 0
    
    # Fissure diagonale sur le carré
    for i in range(70):
        y = 35 + i
        x = 215 + int(i * 0.6)
        if 30 <= y < 130 and 210 <= x < 258:
            img[y, x] = 0
            img[y, min(x+1, 259)] = 0
    
    # Petites fissures/éclats dans le carré
    img[50:52, 225:250] = 0
    img[90:110, 230:233] = 0
    img[70:72, 215:240] = 0
    
    # Petits trous/éclats dispersés dans les lettres
    trous = [(45, 38), (100, 42), (55, 95), (110, 100),  # Dans le H
             (38, 158), (120, 162), (70, 155),            # Dans le I
             (45, 235), (85, 245), (115, 225)]            # Dans le carré
    for (ty, tx) in trous:
        rayon = np.random.randint(2, 4)
        for dy in range(-rayon, rayon+1):
            for dx in range(-rayon, rayon+1):
                if dy*dy + dx*dx <= rayon*rayon:
                    ny, nx = ty + dy, tx + dx
                    if 0 <= ny < h and 0 <= nx < w:
                        img[ny, nx] = 0
    
    return np.clip(img, 0, 255).astype(np.uint8)

img_bh = creer_image_test_blackhat()

es5 = creer_element_structurant('carre', 5)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].imshow(img_bh, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(black_hat(img_bh, es5), cmap='gray'); axes[1].set_title('Black-Hat (sélection des défauts)'); axes[1].axis('off')
plt.tight_layout()
plt.show()

### 5.6 : Exercice - Créer un pipeline de nettoyage

Nous générons une simulation de document bruité :

In [ ]:
# Document bruité
def creer_document_bruite():
    h, w = 200, 300
    doc = np.ones((h, w), dtype=np.uint8) * 240
    for y in range(30, 180, 25):
        for x in range(20, 280, 8):
            if np.random.random() > 0.3:
                doc[y:y+np.random.randint(12,18), x:x+np.random.randint(4,7)] = np.random.randint(20, 60)
    for _ in range(500):
        doc[np.random.randint(0, h), np.random.randint(0, w)] = 0 if np.random.random() > 0.5 else 255
    return doc

doc = creer_document_bruite()
plt.imshow(doc, cmap='gray'); plt.title('Document bruité'); plt.axis('off'); plt.show()

In [ ]:
# EXERCICE : Complétez le pipeline
def nettoyer_document(img):
    # 1. Binarisation
    # 2. Ouverture
    # 3. Fermeture
    pass  # À compléter

In [ ]:
# CORRIGÉ
def nettoyer_document(img):
    seuil = seuil_otsu(img)
    img_bin = np.where(img < seuil, 255, 0).astype(np.uint8)
    es = creer_element_structurant('carre', 3)
    return fermeture(ouverture(img_bin, es), es)

resultat = nettoyer_document(doc)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].imshow(doc, cmap='gray'); axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(resultat, cmap='gray'); axes[1].set_title('Nettoyé'); axes[1].axis('off')
plt.tight_layout()
plt.show()


### Récapitulatif morphologie

| Opération | Formule | Effet |
|-----------|---------|-------|
| **Érosion** | $A \ominus B$ | Amincit |
| **Dilatation** | $A \oplus B$ | Épaissit |
| **Ouverture** | $(A \ominus B) \oplus B$ | Supprime bruit |
| **Fermeture** | $(A \oplus B) \ominus B$ | Remplit trous |
| **Gradient** | Dil - Ero | Contours |



## Suite : TP pour découvrir OpenCV et Scikit-Image

```bash
pip install opencv-python scikit-image
```

## Pour aller plus loin

Notions :
* Filtres non-linéaires : filtre médian, filtre bilatéral (lissage préservant les contours)
* Segmentation avancée : watershed, region growing, graph cuts
* Morphologie mathématique : squelettisation, reconstruction géodésique, ligne de partage des eaux
* Détection de contours : algorithme de Canny en détail, contours actifs (snakes)

Bibliothèques :
* suite au TP : approfondir sa connaissance de OpenCV et Scikit-Image
* SimpleITK : traitement d'images médicales (3D, formats DICOM)
* Pillow (PIL) : manipulation d'images simple et légère
* imageio : lecture/écriture de formats variés (GIF, vidéo)
* Mahotas : alternative à scikit-image avec des fonctions de morphologie avancées

Données :
* traiter des images réelles (médicales, industrielles, de tous les jours…)
* [cours complet](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/intro.html) de traitement d’images biologiques/médicales
